Hola, bienvenido al caso de estudio celumovil store. Aquí, mostraremos una opción para gestionar los inventarios en las diferentes ciudades del país. Esperamos le guste mucho 

Presentado por: Correa Adriana, Rodriguez Jairo, Prieto Kevin, Parra Daniel y Santos Daniel 

4. Ahora viene la parte interesante, ya que tenemos los productos que probablemente gusten en estas ciudades necesitamos predecir que tantas unidades ofertar para dicha ciudad, esto con el fin de saber en cual ciudad centrarnos mas 

In [168]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.linear_model import LinearRegression
import numpy as np
from datetime import datetime
from dateutil.relativedelta import relativedelta
import warnings

# Configuración inicial
warnings.filterwarnings('ignore')

# =============================================
# CONFIGURACIÓN DE PARÁMETROS (AJUSTAR SEGÚN DATOS)
# =============================================
excel_filename = 'basesi.xlsx'  # Nombre del archivo Excel
columna_geografica = 'CIUDAD'   # Columna con ubicaciones (ciudades/departamentos)
columna_producto = 'PRODUCTO'   # Columna con nombres de productos
columna_fecha = 'FECHA'         # Columna con fechas de transacciones
columna_cantidad = 'CANTIDAD'   # Columna con cantidades vendidas (opcional)

min_meses_historia = 2        # Mínimo de meses con datos para hacer predicción
default_cantidad = 10         # Valor por defecto cuando no hay suficiente historia o para productos nuevos

# =============================================
# 1. CARGA Y PREPARACIÓN DE DATOS
# =============================================
print("Cargando y preparando datos...")

try:
    # Cargar datos originales
    data_original = pd.read_excel(excel_filename, engine='openpyxl')
    print(f"Datos cargados. Dimensiones: {data_original.shape}")
    
    # Verificar columnas requeridas
    columnas_requeridas = [columna_geografica, columna_producto, columna_fecha]
    for col in columnas_requeridas:
        if col not in data_original.columns:
            raise ValueError(f"Columna requerida '{col}' no encontrada en el archivo")
    
    # Limpieza básica
    data = data_original[columnas_requeridas + ([columna_cantidad] if columna_cantidad in data_original.columns else [])].copy()
    data.dropna(subset=columnas_requeridas, inplace=True)
    
    # Convertir fecha y extraer mes-año
    data[columna_fecha] = pd.to_datetime(data[columna_fecha], errors='coerce')
    data.dropna(subset=[columna_fecha], inplace=True)
    data['MES_ANO'] = data[columna_fecha].dt.to_period('M')
    
    # Si no hay columna cantidad, asumimos 1 unidad por registro
    # Si existe, llenar NaNs con 1 (o considerar 0 si tiene más sentido para el negocio)
    if columna_cantidad in data.columns:
        data[columna_cantidad] = pd.to_numeric(data[columna_cantidad], errors='coerce').fillna(1)
    else:
        data[columna_cantidad] = 1
        print(f"'{columna_cantidad}' no encontrada, se usará 1 por defecto para cada transacción.")
    
    print("Datos preparados correctamente.")
    print(f"Ubicaciones únicas: {data[columna_geografica].nunique()}")
    print(f"Productos únicos: {data[columna_producto].nunique()}")
    print(f"Rango de fechas: {data[columna_fecha].min()} a {data[columna_fecha].max()}")
    
except Exception as e:
    print(f"Error al cargar/preparar datos: {e}")
    raise

# =============================================
# 2. SISTEMA DE RECOMENDACIÓN (CLUSTERING)
# =============================================
print("\nGenerando recomendaciones por similitud...")

# Crear matriz de interacciones (ubicación x producto)
# Usamos sum() para agregar cantidades si hay múltiples registros del mismo producto en la misma ubicación (implícito por el groupby posterior)
# Aquí, para la matriz de interacción, es mejor usar la presencia/ausencia o una cuenta agregada simple.
# La suma ya se hace para ventas_mensuales. Para la matriz de interacción, la suma de cantidades está bien.
interaction_matrix = data.groupby([columna_geografica, columna_producto])[columna_cantidad].sum().unstack(fill_value=0)

# Calcular similitud coseno entre ubicaciones
if interaction_matrix.shape[0] >= 2:
    lugar_similarity = cosine_similarity(interaction_matrix)
    lugar_sim_df = pd.DataFrame(lugar_similarity, 
                                index=interaction_matrix.index, 
                                columns=interaction_matrix.index)
elif interaction_matrix.shape[0] > 0:
    print("Solo hay una ubicación. No se pueden calcular similitudes entre ubicaciones.")
    lugar_sim_df = pd.DataFrame(index=interaction_matrix.index, columns=interaction_matrix.index) # DataFrame vacío con índice/columnas
else:
    raise ValueError("No hay datos de interacción suficientes para construir la matriz de interacción.")


def obtener_recomendaciones(lugar_actual, n_recomendaciones=5):
    """
    Obtiene recomendaciones de productos para una ubicación basado en ubicaciones similares
    """
    if lugar_actual not in lugar_sim_df.index or lugar_sim_df.empty:
        return []
    
    # Obtener ubicaciones similares (excluyendo la actual)
    # Asegurarse de que hay otras ubicaciones para comparar
    if len(lugar_sim_df.columns) < 2:
        return []
        
    similar_lugares = lugar_sim_df[lugar_actual].sort_values(ascending=False).index[1:]
    
    # Productos que ya tiene la ubicación actual
    productos_actuales = interaction_matrix.loc[lugar_actual][interaction_matrix.loc[lugar_actual] > 0].index.tolist()
    
    # Recolectar recomendaciones con sus scores
    recomendaciones = {}
    for lugar_similar in similar_lugares:
        if lugar_similar == lugar_actual: # Doble chequeo
            continue

        # Productos del lugar similar que no están en el actual
        productos_lugar_similar = interaction_matrix.loc[lugar_similar]
        productos_nuevos = productos_lugar_similar[
            (productos_lugar_similar > 0) & 
            (~productos_lugar_similar.index.isin(productos_actuales))
        ].index.tolist()
        
        # Ponderar por similitud
        sim_score = lugar_sim_df.loc[lugar_similar, lugar_actual] # Similitud entre lugar_similar y lugar_actual
        for producto in productos_nuevos:
            recomendaciones[producto] = recomendaciones.get(producto, 0) + sim_score
    
    # Ordenar y devolver top N
    return sorted(recomendaciones.items(), key=lambda x: x[1], reverse=True)[:n_recomendaciones]

# Generar recomendaciones para todas las ubicaciones
recomendaciones_por_ubicacion = {}
if not interaction_matrix.empty:
    for ubicacion_idx in interaction_matrix.index:
        recs = obtener_recomendaciones(ubicacion_idx)
        if recs:
            recomendaciones_por_ubicacion[ubicacion_idx] = recs

if not recomendaciones_por_ubicacion:
    print("\nNo se pudieron generar recomendaciones (pocas ubicaciones o datos).")
else:
    print("\nRecomendaciones generadas:")
    for ubicacion_rec, recs_list in recomendaciones_por_ubicacion.items():
        print(f"\n{ubicacion_rec}:")
        for producto_rec, score_rec in recs_list:
            print(f"  - {producto_rec} (score: {score_rec:.4f})")

# =============================================
# 3. PREDICCIÓN DE CANTIDADES (REGRESIÓN LINEAL Y HEURÍSTICAS)
# =============================================
print("\n\nPrediciendo cantidades para productos recomendados...")

# Preparar datos históricos mensuales
ventas_mensuales = data.groupby(
    [columna_geografica, columna_producto, 'MES_ANO']
)[columna_cantidad].sum().reset_index()
ventas_mensuales['MES_ANO'] = ventas_mensuales['MES_ANO'].dt.to_timestamp()

# Calcular la venta mensual promedio de cada producto en las ubicaciones donde se vende
# Esto servirá como base para predecir en nuevas ubicaciones.
promedio_ventas_producto_global = ventas_mensuales.groupby(columna_producto)[columna_cantidad].mean().reset_index()
promedio_ventas_producto_global.rename(columns={columna_cantidad: 'CANTIDAD_PROMEDIO_GLOBAL'}, inplace=True)

predicciones_finales = {}

for ubicacion, productos_recomendados in recomendaciones_por_ubicacion.items():
    predicciones_finales[ubicacion] = []
    
    for producto, score in productos_recomendados:
        # Filtrar datos históricos para este producto y ubicación actual
        historial_local = ventas_mensuales[
            (ventas_mensuales[columna_geografica] == ubicacion) &
            (ventas_mensuales[columna_producto] == producto)
        ].sort_values('MES_ANO')
        
        cantidad_predicha = default_cantidad  # Valor por defecto inicial
        meses_historicos_locales = len(historial_local)
        
        if meses_historicos_locales >= min_meses_historia:
            # Hay suficiente historia local, usar regresión lineal
            historial_local = historial_local.assign(
                TIME_INDEX=range(len(historial_local))
            )
            try:
                model = LinearRegression()
                model.fit(historial_local[['TIME_INDEX']], historial_local[columna_cantidad])
                
                next_index = [[len(historial_local)]] # Predecir para el siguiente período
                pred = model.predict(next_index)[0]
                
                cantidad_predicha = max(1, round(pred)) # Asegurar valor positivo, mínimo 1
                
                # Limitar predicciones muy altas usando el percentil 75 del histórico local
                # Solo aplicar si el percentil es positivo para evitar problemas con historiales de ceros.
                p75_local = historial_local[columna_cantidad].quantile(0.75)
                if p75_local > 0 and cantidad_predicha > 3 * p75_local:
                    cantidad_predicha = max(default_cantidad, round(p75_local))
                elif p75_local == 0 and cantidad_predicha > 2 * default_cantidad: # Historial bajo pero predicción alta
                    cantidad_predicha = 2 * default_cantidad

            except Exception as e_reg:
                # print(f"Advertencia: Falla en regresión para {ubicacion}-{producto}: {e_reg}. Usando mediana local.")
                # Si la regresión falla, usar la mediana histórica local, asegurando que sea al menos default_cantidad
                cantidad_predicha = max(default_cantidad, round(historial_local[columna_cantidad].median()))
                cantidad_predicha = max(1, cantidad_predicha) # Asegurar mínimo 1
        
        elif meses_historicos_locales > 0: # Poca historia local (menos que min_meses_historia pero > 0)
            # print(f"Advertencia: Poca historia local para {ubicacion}-{producto}. Usando media local.")
            # Usar promedio histórico local, asegurando que sea al menos default_cantidad
            cantidad_predicha = max(default_cantidad, round(historial_local[columna_cantidad].mean()))
            cantidad_predicha = max(1, cantidad_predicha) # Asegurar mínimo 1
        
        else: # No hay historia local (producto verdaderamente nuevo para la ubicación)
            # print(f"Info: Sin historia local para {ubicacion}-{producto}. Usando promedio de otras ubicaciones o default.")
            # Buscar la venta promedio de este producto en otras ubicaciones
            info_producto_otras_ubic = promedio_ventas_producto_global[
                promedio_ventas_producto_global[columna_producto] == producto
            ]
            
            if not info_producto_otras_ubic.empty:
                cantidad_promedio_global_producto = info_producto_otras_ubic['CANTIDAD_PROMEDIO_GLOBAL'].iloc[0]
                # Usar el promedio global del producto, pero no menos que default_cantidad.
                # Esto significa que si el producto generalmente se vende poco (ej., 3 unidades),
                # y default_cantidad es 10, se predecirán 10.
                # Si se vende bien globalmente (ej., 20 unidades), se predecirán 20.
                cantidad_predicha = max(default_cantidad, round(cantidad_promedio_global_producto))
                cantidad_predicha = max(1, cantidad_predicha) # Asegurar mínimo 1
            else:
                # El producto no tiene historial en ninguna otra ubicación (producto nuevo en general)
                # print(f"Info: Producto {producto} sin historial global. Usando default_cantidad.")
                cantidad_predicha = default_cantidad # Ya es max(1, default_cantidad) implicitamente si default_cantidad >=1
        
        predicciones_finales[ubicacion].append({
            'Producto': producto,
            'Score_Recomendacion': score,
            'Cantidad_Predicha': int(cantidad_predicha), # Asegurar que la cantidad sea entera
            
        })

# =============================================
# 4. RESULTADOS FINALES
# =============================================
print("\n\n=== RESULTADOS FINALES ===")
if not predicciones_finales:
    print(f"No se generaron predicciones finales.")
else:
    print(f"Recomendaciones con predicciones de cantidad (mínimo {max(1,default_cantidad)} unidades si no hay otra info)")

    for ubicacion_res, predicciones_list in predicciones_finales.items():
        print(f"\n--- {ubicacion_res} ---")
        if predicciones_list:
            df_res = pd.DataFrame(predicciones_list)
            df_res['Score_Recomendacion'] = df_res['Score_Recomendacion'].round(4)
            # Cambiado 'Meses_Historicos' a 'Meses_Historicos_Locales'
            print(df_res[['Producto', 'Score_Recomendacion', 'Cantidad_Predicha']].to_string(index=False))
        else:
            print("No hay predicciones para esta ubicación.")

Cargando y preparando datos...
Datos cargados. Dimensiones: (57429, 68)
'CANTIDAD' no encontrada, se usará 1 por defecto para cada transacción.
Datos preparados correctamente.
Ubicaciones únicas: 805
Productos únicos: 614
Rango de fechas: 1970-01-01 00:00:00.001022024 a 1970-01-01 00:00:00.031122024

Generando recomendaciones por similitud...

Recomendaciones generadas:

Abejorral / Antioquia:
  - FLETE POR ENVIO (score: 47.1837)
  - CELULAR NOTE 13 PRO + 5G 512 NEGRO 12RAM (score: 46.1521)
  - CELULAR POCO X6 PRO 5G 512 AMARILLO (score: 44.6954)
  - CELULAR POCO X6 PRO 5G 512 GRIS (score: 41.4118)
  - CELULAR SAMSUNG A55 5G 256 AZUL OSCURO 8 (score: 36.9028)

Abrego / Norte de Santander:
  - CELULAR POCO X6 PRO 5G 512 NEGRO 12 (score: 57.4650)
  - CELULAR POCO X6 PRO 5G 512 AMARILLO (score: 55.7345)
  - CELULAR SAMSUNG A55 5G 256 AZUL 8G (score: 45.1697)
  - CELULAR NOTE 13 PRO 5G 256 NEGRO 8 RAM (score: 44.2462)
  - CELULAR SAMSUNG A55 5G 256 AZUL OSCURO 8 (score: 44.1197)

Acacías /

In [170]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.linear_model import LinearRegression
import numpy as np
from datetime import datetime
# from dateutil.relativedelta import relativedelta # No se usa explícitamente, se puede quitar si no hay planes futuros
import warnings

# Configuración inicial
warnings.filterwarnings('ignore')

# =============================================
# CONFIGURACIÓN DE PARÁMETROS (AJUSTAR SEGÚN DATOS)
# =============================================
excel_filename = 'basesi.xlsx'
columna_geografica = 'CIUDAD'
columna_producto = 'PRODUCTO'
columna_fecha = 'FECHA'
columna_cantidad = 'CANTIDAD'

min_meses_historia = 2
default_cantidad = 10
min_meses_para_regresion = 4
top_n_similar_locations_for_cold_start = 5

# =============================================
# 1. CARGA Y PREPARACIÓN DE DATOS
# =============================================
print("Cargando y preparando datos...")
try:
    data_original = pd.read_excel(excel_filename, engine='openpyxl')
    print(f"Datos cargados. Dimensiones: {data_original.shape}")
    
    columnas_requeridas_base = [columna_geografica, columna_producto, columna_fecha]
    columnas_a_usar = columnas_requeridas_base.copy()
    if columna_cantidad in data_original.columns:
        columnas_a_usar.append(columna_cantidad)
    else:
        print(f"Advertencia: '{columna_cantidad}' no encontrada. Se asumirá 1 por transacción.")

    for col in columnas_requeridas_base:
        if col not in data_original.columns:
            raise ValueError(f"Columna requerida '{col}' no encontrada en el archivo")
    
    data = data_original[columnas_a_usar].copy()
    data.dropna(subset=columnas_requeridas_base, inplace=True)
    
    data[columna_fecha] = pd.to_datetime(data[columna_fecha], errors='coerce')
    data.dropna(subset=[columna_fecha], inplace=True)
    data['MES_ANO'] = data[columna_fecha].dt.to_period('M')
    
    if columna_cantidad in data.columns:
        data[columna_cantidad] = pd.to_numeric(data[columna_cantidad], errors='coerce').fillna(1)
        data[columna_cantidad] = data[columna_cantidad].apply(lambda x: max(0, x)) # No cantidades negativas
    else:
        data[columna_cantidad] = 1
    
    print("Datos preparados correctamente.")
except Exception as e:
    print(f"Error al cargar/preparar datos: {e}")
    raise

# =============================================
# 2. SISTEMA DE RECOMENDACIÓN (CLUSTERING)
# =============================================
print("\nGenerando recomendaciones por similitud...")
interaction_matrix = data.groupby([columna_geografica, columna_producto])[columna_cantidad].sum().unstack(fill_value=0)

lugar_sim_df = None
if interaction_matrix.shape[0] >= 2:
    lugar_similarity_values = cosine_similarity(interaction_matrix)
    lugar_sim_df = pd.DataFrame(lugar_similarity_values, 
                                index=interaction_matrix.index, 
                                columns=interaction_matrix.index)
elif not interaction_matrix.empty:
    print("Advertencia: Solo hay una ubicación o ninguna. No se pueden calcular similitudes entre ubicaciones para recomendaciones detalladas.")
    idx = interaction_matrix.index
    if len(idx) > 0: # Si hay una ubicación
        lugar_sim_df = pd.DataFrame(np.eye(len(idx)), index=idx, columns=idx) # Matriz identidad para evitar errores
    else: # Si no hay ubicaciones (interaction_matrix vacía)
         lugar_sim_df = pd.DataFrame() # DataFrame vacío
else:
    print("Error: No hay datos de interacción suficientes. La matriz de interacción está vacía.")
    lugar_sim_df = pd.DataFrame() # DataFrame vacío

# OPTIMIZACIÓN: Pre-calcular las N ubicaciones más similares para cada ubicación
top_similar_partners = {}
if lugar_sim_df is not None and not lugar_sim_df.empty:
    for loc_idx_sim_calc in lugar_sim_df.index:
        if loc_idx_sim_calc in lugar_sim_df.columns: # Sanity check
            sim_scores = lugar_sim_df[loc_idx_sim_calc].drop(loc_idx_sim_calc, errors='ignore').sort_values(ascending=False)
            top_similar_partners[loc_idx_sim_calc] = sim_scores.head(top_n_similar_locations_for_cold_start).index.tolist()
        else:
            top_similar_partners[loc_idx_sim_calc] = []

def obtener_recomendaciones(lugar_actual, n_recomendaciones=5):
    if lugar_sim_df is None or lugar_actual not in lugar_sim_df.index or lugar_sim_df.empty:
        return []
    # Si no hay otras ubicaciones para comparar (solo 1 ubicación en total)
    if len(lugar_sim_df.columns) < 2 :
         return []

    similar_lugares_scores = lugar_sim_df[lugar_actual].sort_values(ascending=False)
    similar_lugares = [loc for loc in similar_lugares_scores.index if loc != lugar_actual] # Excluirse a sí mismo

    if not similar_lugares or lugar_actual not in interaction_matrix.index: # Si no hay lugares similares o el lugar actual no está en la matriz
        return []

    productos_actuales = interaction_matrix.loc[lugar_actual][interaction_matrix.loc[lugar_actual] > 0].index.tolist()
    
    recomendaciones = {}
    for lugar_similar in similar_lugares:
        if lugar_similar not in interaction_matrix.index: continue
        
        productos_lugar_similar = interaction_matrix.loc[lugar_similar]
        productos_nuevos = productos_lugar_similar[
            (productos_lugar_similar > 0) & 
            (~productos_lugar_similar.index.isin(productos_actuales))
        ].index.tolist()
        
        sim_score = similar_lugares_scores.get(lugar_similar, 0) # Usar .get para seguridad
        for producto in productos_nuevos:
            recomendaciones[producto] = recomendaciones.get(producto, 0) + sim_score
    
    return sorted(recomendaciones.items(), key=lambda x: x[1], reverse=True)[:n_recomendaciones]

recomendaciones_por_ubicacion = {}
if not interaction_matrix.empty:
    for ubicacion_idx in interaction_matrix.index:
        recs = obtener_recomendaciones(ubicacion_idx)
        if recs:
            recomendaciones_por_ubicacion[ubicacion_idx] = recs

if not recomendaciones_por_ubicacion:
    print("\nNo se pudieron generar recomendaciones.")
else:
    print("\nRecomendaciones generadas.")

# =============================================
# 3. PREDICCIÓN DE CANTIDADES (LÓGICA MEJORADA Y OPTIMIZADA)
# =============================================
print("\n\nPrediciendo cantidades para productos recomendados con lógica mejorada y optimizada...")

ventas_mensuales_df = data.groupby(
    [columna_geografica, columna_producto, 'MES_ANO']
)[columna_cantidad].sum().reset_index()
ventas_mensuales_df['MES_ANO'] = ventas_mensuales_df['MES_ANO'].dt.to_timestamp()

# OPTIMIZACIÓN: Crear versión indexada para búsquedas rápidas de historial_local
if not ventas_mensuales_df.empty:
    ventas_mensuales_indexed = ventas_mensuales_df.set_index([columna_geografica, columna_producto])
else: # Si no hay ventas, crear un DF indexado vacío con las columnas esperadas
    ventas_mensuales_indexed = pd.DataFrame(columns=['MES_ANO', columna_cantidad]).set_index(pd.MultiIndex.from_tuples([], names=(columna_geografica, columna_producto)))


promedio_ventas_producto_global_df = ventas_mensuales_df.groupby(columna_producto, as_index=False)[columna_cantidad].mean()
promedio_ventas_producto_global_df.rename(columns={columna_cantidad: 'CANTIDAD_PROMEDIO_GLOBAL'}, inplace=True)

predicciones_finales = {}

def get_avg_sales_in_locations(product_name, locations_list, ventas_df_local, geo_col, prod_col, qty_col):
    if not locations_list or ventas_df_local.empty:
        return None
    relevant_sales = ventas_df_local[
        (ventas_df_local[prod_col] == product_name) &
        (ventas_df_local[geo_col].isin(locations_list))
    ]
    if not relevant_sales.empty and qty_col in relevant_sales.columns:
        return relevant_sales[qty_col].mean()
    return None

for ubicacion, productos_recomendados in recomendaciones_por_ubicacion.items():
    predicciones_finales[ubicacion] = []
    
    for producto, score in productos_recomendados:
        historial_local = pd.DataFrame() # Inicializar como DataFrame vacío
        try:
            # Usar el DataFrame indexado
            historial_data_lookup = ventas_mensuales_indexed.loc[(ubicacion, producto)]
            if isinstance(historial_data_lookup, pd.Series):
                historial_local = historial_data_lookup.to_frame().T.sort_values('MES_ANO').reset_index()
            elif isinstance(historial_data_lookup, pd.DataFrame):
                historial_local = historial_data_lookup.sort_values('MES_ANO').reset_index()
        except KeyError:
            # El par (ubicacion, producto) no tiene historial, historial_local permanece vacío
            pass
        # Asegurar que las columnas esperadas existan incluso si está vacío
        if historial_local.empty:
             historial_local = pd.DataFrame(columns=[columna_geografica, columna_producto, 'MES_ANO', columna_cantidad])


        cantidad_predicha_float = float(default_cantidad) # Usar float para cálculos intermedios
        meses_historicos_locales = len(historial_local)
        
        if meses_historicos_locales >= min_meses_para_regresion:
            historial_local_reg = historial_local.assign(TIME_INDEX=np.arange(len(historial_local)))
            try:
                model = LinearRegression()
                model.fit(historial_local_reg[['TIME_INDEX']], historial_local_reg[columna_cantidad])
                pred = model.predict([[len(historial_local_reg)]])[0]
                cantidad_predicha_float = pred
                
                p75_local = historial_local_reg[columna_cantidad].quantile(0.75)
                if pd.notna(p75_local) and p75_local > 0 and cantidad_predicha_float > 3 * p75_local:
                    mean_local = historial_local_reg[columna_cantidad].mean()
                    std_local = historial_local_reg[columna_cantidad].std()
                    cap_val = 1.5 * p75_local
                    if pd.notna(mean_local) and pd.notna(std_local):
                        cap_val = max(cap_val, mean_local + 2 * std_local)
                    if pd.notna(cap_val): cantidad_predicha_float = cap_val
                elif (pd.isna(p75_local) or p75_local == 0) and cantidad_predicha_float > default_cantidad * 2:
                     cantidad_predicha_float = float(default_cantidad * 2)
            except Exception: # Falla en regresión
                if not historial_local.empty and columna_cantidad in historial_local.columns and pd.notna(historial_local[columna_cantidad].mean()):
                    cantidad_predicha_float = historial_local[columna_cantidad].mean()
        
        elif meses_historicos_locales > 0: # Poca historia
            if not historial_local.empty and columna_cantidad in historial_local.columns and pd.notna(historial_local[columna_cantidad].mean()):
                 cantidad_predicha_float = historial_local[columna_cantidad].mean()

        else: # Sin historia local (arranque en frío)
            estimacion_cold_start = None
            # Usar top_similar_partners precalculado
            top_sim_locs = top_similar_partners.get(ubicacion, [])
            if top_sim_locs:
                avg_sales_sim = get_avg_sales_in_locations(
                    producto, top_sim_locs, ventas_mensuales_df, # Pasar el DF original no indexado
                    columna_geografica, columna_producto, columna_cantidad
                )
                if avg_sales_sim is not None and pd.notna(avg_sales_sim):
                    estimacion_cold_start = avg_sales_sim
            
            if estimacion_cold_start is None or estimacion_cold_start < 1:
                info_prod_gbl = promedio_ventas_producto_global_df[
                    promedio_ventas_producto_global_df[columna_producto] == producto
                ]
                if not info_prod_gbl.empty:
                    avg_gbl_sales = info_prod_gbl['CANTIDAD_PROMEDIO_GLOBAL'].iloc[0]
                    if pd.notna(avg_gbl_sales):
                         estimacion_cold_start = avg_gbl_sales
            
            if estimacion_cold_start is not None and pd.notna(estimacion_cold_start) and estimacion_cold_start >= 1:
                cantidad_predicha_float = estimacion_cold_start
            # else: cantidad_predicha_float permanece default_cantidad

        # Redondeo final y asegurar mínimo 1
        cantidad_predicha = max(1, int(round(cantidad_predicha_float)))

        predicciones_finales[ubicacion].append({
            'Producto': producto,
            'Score_Recomendacion': score,
            'Cantidad_Predicha': cantidad_predicha,
            'Meses_Historicos_Locales': meses_historicos_locales
        })

# =============================================
# 4. RESULTADOS FINALES
# =============================================
print("\n\n=== RESULTADOS FINALES ===")
if not predicciones_finales:
    print(f"No se generaron predicciones finales.")
else:
    print(f"Recomendaciones con predicciones de cantidad:")
    for ubicacion_res, predicciones_list in predicciones_finales.items():
        print(f"\n--- {ubicacion_res} ---")
        if predicciones_list:
            df_res = pd.DataFrame(predicciones_list)
            if not df_res.empty :
                df_res['Score_Recomendacion'] = df_res['Score_Recomendacion'].round(4)
                print(df_res[['Producto', 'Score_Recomendacion', 'Cantidad_Predicha', 'Meses_Historicos_Locales']].to_string(index=False))
        else:
            print("No hay predicciones para esta ubicación.")

Cargando y preparando datos...
Datos cargados. Dimensiones: (57429, 68)
Advertencia: 'CANTIDAD' no encontrada. Se asumirá 1 por transacción.
Datos preparados correctamente.

Generando recomendaciones por similitud...

Recomendaciones generadas.


Prediciendo cantidades para productos recomendados con lógica mejorada y optimizada...


=== RESULTADOS FINALES ===
Recomendaciones con predicciones de cantidad:

--- Abejorral / Antioquia ---
                                Producto  Score_Recomendacion  Cantidad_Predicha  Meses_Historicos_Locales
                         FLETE POR ENVIO              47.1837                  1                         0
CELULAR NOTE 13 PRO + 5G 512 NEGRO 12RAM              46.1521                  1                         0
     CELULAR POCO X6 PRO 5G 512 AMARILLO              44.6954                  1                         0
         CELULAR POCO X6 PRO 5G 512 GRIS              41.4118                  7                         0
CELULAR SAMSUNG A55 5G 25

In [174]:
print(df.columns)


Index(['#', 'DOC-#', 'FECHA', 'PRODUCTO', 'CLIENTE', 'CIUDAD', 'OFICIAL',
       'NETO', '% P', '% A', 'FP', 'FECHA.1', 'VOUCHER', 'CÓDIGO', 'NOMBRE',
       'V-NETO', 'V-NETO+IVA', 'COSTO ACTUAL', 'MACROCLIENTE-2', ' ', ' .1',
       ' .2', ' .3', ' .4', ' .5', ' .6', ' .7', ' .8', ' .9', ' .10', ' .11',
       ' .12', ' .13', ' .14', ' .15', ' .16', ' .17', ' .18', ' .19', ' .20',
       ' .21', ' .22', ' .23', ' .24', ' .25', ' .26', ' .27', ' .28', ' .29',
       ' .30', ' .31', ' .32', ' .33', ' .34', ' .35', ' .36', ' .37', ' .38',
       ' .39', ' .40', ' .41', ' .42', ' .43', ' .44', ' .45', ' .46', ' .47',
       ' .48'],
      dtype='object')


In [188]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.linear_model import LinearRegression
import numpy as np
from datetime import datetime
# from dateutil.relativedelta import relativedelta # No se usa explícitamente, se puede quitar si no hay planes futuros
import warnings

# Configuración inicial
warnings.filterwarnings('ignore')

# =============================================
# CONFIGURACIÓN DE PARÁMETROS (AJUSTAR SEGÚN DATOS)
# =============================================
excel_filename = 'basesi.xlsx' # Asegúrate de que este sea el nombre correcto de tu archivo Excel
columna_geografica = 'CIUDAD'   # Nombre de tu columna para la ubicación geográfica
columna_producto = 'PRODUCTO'  # Nombre de tu columna para el producto
columna_fecha = 'FECHA'      # Nombre de tu columna para la fecha
columna_cantidad = 'CANTIDAD'  # Nombre de tu columna para la cantidad vendida (si es diferente, ajústala aquí)

min_meses_historia = 2
default_cantidad = 10
min_meses_para_regresion = 4
top_n_similar_locations_for_cold_start = 5

# =============================================
# 1. CARGA Y PREPARACIÓN DE DATOS
# =============================================
print("Cargando y preparando datos...")
try:
    data_original = pd.read_excel(excel_filename, engine='openpyxl')
    print(f"Datos cargados. Dimensiones: {data_original.shape}")
    
    # CORRECCIÓN: Usar la variable correcta 'columna_fecha'
    columnas_requeridas_base = [columna_geografica, columna_producto, columna_fecha]
    columnas_a_usar = columnas_requeridas_base.copy()
    
    if columna_cantidad in data_original.columns:
        columnas_a_usar.append(columna_cantidad)
    else:
        print(f"Advertencia: La columna de cantidad '{columna_cantidad}' no fue encontrada. Se asumirá 1 por transacción.")
        # Si la columna cantidad no existe, la crearemos y la llenaremos con 1 más adelante.

    for col in columnas_requeridas_base:
        if col not in data_original.columns:
            raise ValueError(f"Columna requerida '{col}' no encontrada en el archivo Excel. Verifica los nombres en la configuración.")
    
    data = data_original[columnas_a_usar].copy()
    data.dropna(subset=columnas_requeridas_base, inplace=True) # Eliminar filas donde las columnas base son NaN
    
    # Convertir columna de fecha y manejar errores
    data[columna_fecha] = pd.to_datetime(data[columna_fecha], errors='coerce')
    data.dropna(subset=[columna_fecha], inplace=True) # Eliminar filas con fechas inválidas
    data['MES_ANO'] = data[columna_fecha].dt.to_period('M')
    
    # Procesar columna de cantidad
    if columna_cantidad in data.columns:
        data[columna_cantidad] = pd.to_numeric(data[columna_cantidad], errors='coerce').fillna(1)
        data[columna_cantidad] = data[columna_cantidad].apply(lambda x: max(0, float(x))) # No cantidades negativas y asegurar que sea float
    else:
        # Si la columna cantidad no estaba, la creamos aquí con valor 1
        data[columna_cantidad] = 1.0
    
    print("Datos preparados correctamente.")
except FileNotFoundError:
    print(f"Error: El archivo '{excel_filename}' no fue encontrado. Por favor, verifica el nombre y la ruta del archivo.")
    raise
except ValueError as ve:
    print(f"Error de valor durante la carga/preparación: {ve}")
    raise
except Exception as e:
    print(f"Ocurrió un error inesperado al cargar/preparar datos: {e}")
    raise

# =============================================
# 2. SISTEMA DE RECOMENDACIÓN (CLUSTERING)
# =============================================
print("\nGenerando recomendaciones por similitud...")
interaction_matrix = pd.DataFrame()
if not data.empty:
    interaction_matrix = data.groupby([columna_geografica, columna_producto])[columna_cantidad].sum().unstack(fill_value=0)
else:
    print("Advertencia: El DataFrame 'data' está vacío después de la preparación. No se puede generar la matriz de interacción.")


lugar_sim_df = None
if not interaction_matrix.empty and interaction_matrix.shape[0] >= 2:
    lugar_similarity_values = cosine_similarity(interaction_matrix)
    lugar_sim_df = pd.DataFrame(lugar_similarity_values, 
                                index=interaction_matrix.index, 
                                columns=interaction_matrix.index)
elif not interaction_matrix.empty:
    print("Advertencia: Solo hay una ubicación en la matriz de interacción o ninguna con suficientes datos. No se pueden calcular similitudes detalladas entre ubicaciones.")
    idx = interaction_matrix.index
    if len(idx) > 0: # Si hay al menos una ubicación
        lugar_sim_df = pd.DataFrame(np.eye(len(idx)), index=idx, columns=idx) # Matriz identidad para evitar errores
    else: 
        lugar_sim_df = pd.DataFrame() 
else:
    print("Error: No hay datos de interacción suficientes. La matriz de interacción está vacía o no se pudo generar.")
    lugar_sim_df = pd.DataFrame()

# OPTIMIZACIÓN: Pre-calcular las N ubicaciones más similares para cada ubicación
top_similar_partners = {}
if lugar_sim_df is not None and not lugar_sim_df.empty:
    for loc_idx_sim_calc in lugar_sim_df.index:
        if loc_idx_sim_calc in lugar_sim_df.columns: 
            sim_scores = lugar_sim_df[loc_idx_sim_calc].drop(loc_idx_sim_calc, errors='ignore').sort_values(ascending=False)
            top_similar_partners[loc_idx_sim_calc] = sim_scores.head(top_n_similar_locations_for_cold_start).index.tolist()
        else:
            top_similar_partners[loc_idx_sim_calc] = []

def obtener_recomendaciones(lugar_actual, n_recomendaciones=5):
    if lugar_sim_df is None or lugar_sim_df.empty or lugar_actual not in lugar_sim_df.index:
        return []
    if len(lugar_sim_df.columns) < 2 and len(lugar_sim_df.index) < 2 : # Necesita al menos otro lugar para comparar
        return []

    # Asegurarse de que lugar_actual está en las columnas de similitud también
    if lugar_actual not in lugar_sim_df.columns:
        return []
        
    similar_lugares_scores = lugar_sim_df[lugar_actual].sort_values(ascending=False)
    similar_lugares = [loc for loc in similar_lugares_scores.index if loc != lugar_actual] 

    if not similar_lugares or lugar_actual not in interaction_matrix.index: 
        return []

    productos_actuales = interaction_matrix.loc[lugar_actual][interaction_matrix.loc[lugar_actual] > 0].index.tolist()
    
    recomendaciones = {}
    for lugar_similar in similar_lugares:
        if lugar_similar not in interaction_matrix.index: continue # Saltar si el lugar similar no está en la matriz de interacciones
        
        productos_lugar_similar = interaction_matrix.loc[lugar_similar]
        productos_nuevos = productos_lugar_similar[
            (productos_lugar_similar > 0) & 
            (~productos_lugar_similar.index.isin(productos_actuales))
        ].index.tolist()
        
        sim_score = similar_lugares_scores.get(lugar_similar, 0) 
        for producto in productos_nuevos:
            recomendaciones[producto] = recomendaciones.get(producto, 0) + sim_score
    
    return sorted(recomendaciones.items(), key=lambda x: x[1], reverse=True)[:n_recomendaciones]

recomendaciones_por_ubicacion = {}
if not interaction_matrix.empty and lugar_sim_df is not None and not lugar_sim_df.empty:
    for ubicacion_idx in interaction_matrix.index:
        recs = obtener_recomendaciones(ubicacion_idx)
        if recs:
            recomendaciones_por_ubicacion[ubicacion_idx] = recs

if not recomendaciones_por_ubicacion:
    print("\nNo se pudieron generar recomendaciones basadas en similitud.")
else:
    print("\nRecomendaciones por similitud generadas.")

# =============================================
# 3. PREDICCIÓN DE CANTIDADES (LÓGICA MEJORADA Y OPTIMIZADA)
# =============================================
print("\nPrediciendo cantidades para productos recomendados...")

ventas_mensuales_df = pd.DataFrame()
if not data.empty:
    ventas_mensuales_df = data.groupby(
        [columna_geografica, columna_producto, 'MES_ANO']
    )[columna_cantidad].sum().reset_index()
    if 'MES_ANO' in ventas_mensuales_df.columns:
         ventas_mensuales_df['MES_ANO'] = ventas_mensuales_df['MES_ANO'].dt.to_timestamp()
else:
    print("Advertencia: El DataFrame 'data' está vacío. No se pueden calcular ventas mensuales.")


ventas_mensuales_indexed = pd.DataFrame()
if not ventas_mensuales_df.empty:
    ventas_mensuales_indexed = ventas_mensuales_df.set_index([columna_geografica, columna_producto])
else: 
    ventas_mensuales_indexed = pd.DataFrame(columns=['MES_ANO', columna_cantidad]).set_index(pd.MultiIndex.from_tuples([], names=(columna_geografica, columna_producto)))

promedio_ventas_producto_global_df = pd.DataFrame()
if not ventas_mensuales_df.empty:
    promedio_ventas_producto_global_df = ventas_mensuales_df.groupby(columna_producto, as_index=False)[columna_cantidad].mean()
    promedio_ventas_producto_global_df.rename(columns={columna_cantidad: 'CANTIDAD_PROMEDIO_GLOBAL'}, inplace=True)
else:
    promedio_ventas_producto_global_df = pd.DataFrame(columns=[columna_producto, 'CANTIDAD_PROMEDIO_GLOBAL'])


predicciones_finales = {}

def get_avg_sales_in_locations(product_name, locations_list, ventas_df_local, geo_col, prod_col, qty_col):
    if not locations_list or ventas_df_local.empty:
        return None
    relevant_sales = ventas_df_local[
        (ventas_df_local[prod_col] == product_name) &
        (ventas_df_local[geo_col].isin(locations_list))
    ]
    if not relevant_sales.empty and qty_col in relevant_sales.columns and not relevant_sales[qty_col].empty:
        return relevant_sales[qty_col].mean()
    return None

for ubicacion, productos_recomendados in recomendaciones_por_ubicacion.items():
    predicciones_finales[ubicacion] = []
    
    for producto, score in productos_recomendados:
        historial_local = pd.DataFrame() 
        try:
            if not ventas_mensuales_indexed.empty and (ubicacion, producto) in ventas_mensuales_indexed.index:
                historial_data_lookup = ventas_mensuales_indexed.loc[(ubicacion, producto)]
                if isinstance(historial_data_lookup, pd.Series): # Un solo mes de historial
                    historial_local = historial_data_lookup.to_frame().T.sort_values('MES_ANO').reset_index()
                elif isinstance(historial_data_lookup, pd.DataFrame): # Múltiples meses
                    historial_local = historial_data_lookup.sort_values('MES_ANO').reset_index()
        except KeyError:
            pass 
        
        if historial_local.empty:
            historial_local = pd.DataFrame(columns=[columna_geografica, columna_producto, 'MES_ANO', columna_cantidad])


        cantidad_predicha_float = float(default_cantidad) 
        meses_historicos_locales = len(historial_local)
        
        if meses_historicos_locales >= min_meses_para_regresion:
            historial_local_reg = historial_local.assign(TIME_INDEX=np.arange(len(historial_local)))
            # Asegurarse que la columna cantidad es numérica y no tiene NaNs para la regresión
            if pd.api.types.is_numeric_dtype(historial_local_reg[columna_cantidad]) and not historial_local_reg[columna_cantidad].isnull().any():
                try:
                    model = LinearRegression()
                    model.fit(historial_local_reg[['TIME_INDEX']], historial_local_reg[columna_cantidad])
                    pred = model.predict([[len(historial_local_reg)]])[0]
                    cantidad_predicha_float = pred
                    
                    p75_local = historial_local_reg[columna_cantidad].quantile(0.75)
                    if pd.notna(p75_local) and p75_local > 0 and cantidad_predicha_float > 3 * p75_local:
                        mean_local = historial_local_reg[columna_cantidad].mean()
                        std_local = historial_local_reg[columna_cantidad].std()
                        cap_val = 1.5 * p75_local
                        if pd.notna(mean_local) and pd.notna(std_local): # std_local puede ser NaN si hay solo 1 punto
                            cap_val = max(cap_val, mean_local + 2 * (std_local if pd.notna(std_local) else 0))
                        if pd.notna(cap_val): cantidad_predicha_float = cap_val
                    elif (pd.isna(p75_local) or p75_local == 0) and cantidad_predicha_float > default_cantidad * 2:
                        cantidad_predicha_float = float(default_cantidad * 2)
                except Exception as reg_ex: 
                    print(f"Advertencia: Falla en regresión para {ubicacion}-{producto}. Usando promedio local si existe. Error: {reg_ex}")
                    if not historial_local.empty and columna_cantidad in historial_local.columns and not historial_local[columna_cantidad].empty and pd.notna(historial_local[columna_cantidad].mean()):
                        cantidad_predicha_float = historial_local[columna_cantidad].mean()
            else:
                print(f"Advertencia: Datos insuficientes o no numéricos para regresión en {ubicacion}-{producto}. Usando promedio local.")
                if not historial_local.empty and columna_cantidad in historial_local.columns and not historial_local[columna_cantidad].empty and pd.notna(historial_local[columna_cantidad].mean()):
                    cantidad_predicha_float = historial_local[columna_cantidad].mean()

        elif meses_historicos_locales > 0: 
            if not historial_local.empty and columna_cantidad in historial_local.columns and not historial_local[columna_cantidad].empty and pd.notna(historial_local[columna_cantidad].mean()):
                cantidad_predicha_float = historial_local[columna_cantidad].mean()

        else: # Sin historia local (arranque en frío)
            estimacion_cold_start = None
            top_sim_locs = top_similar_partners.get(ubicacion, [])
            if top_sim_locs:
                avg_sales_sim = get_avg_sales_in_locations(
                    producto, top_sim_locs, ventas_mensuales_df, 
                    columna_geografica, columna_producto, columna_cantidad
                )
                if avg_sales_sim is not None and pd.notna(avg_sales_sim):
                    estimacion_cold_start = avg_sales_sim
            
            if estimacion_cold_start is None or estimacion_cold_start < 1:
                if not promedio_ventas_producto_global_df.empty:
                    info_prod_gbl = promedio_ventas_producto_global_df[
                        promedio_ventas_producto_global_df[columna_producto] == producto
                    ]
                    if not info_prod_gbl.empty:
                        avg_gbl_sales = info_prod_gbl['CANTIDAD_PROMEDIO_GLOBAL'].iloc[0]
                        if pd.notna(avg_gbl_sales):
                            estimacion_cold_start = avg_gbl_sales
            
            if estimacion_cold_start is not None and pd.notna(estimacion_cold_start) and estimacion_cold_start >= 1:
                cantidad_predicha_float = estimacion_cold_start
        
        # Redondeo final y asegurar mínimo 1 y que sea positivo
        cantidad_predicha_float = max(0, cantidad_predicha_float) # Asegurar que no sea negativo antes de redondear
        cantidad_predicha = max(1, int(round(cantidad_predicha_float)))


        predicciones_finales[ubicacion].append({
            'Producto': producto,
            'Score_Recomendacion': score,
            'Cantidad_Predicha': cantidad_predicha,
            'Meses_Historicos_Locales': meses_historicos_locales
        })

# =============================================
# 4. RESULTADOS FINALES
# =============================================
print("\n\n=== RESULTADOS FINALES ===")
if not predicciones_finales:
    print(f"No se generaron predicciones finales.")
else:
    print(f"Recomendaciones con predicciones de cantidad:")
    for ubicacion_res, predicciones_list in predicciones_finales.items():
        print(f"\n--- {ubicacion_res} ---")
        if predicciones_list:
            df_res = pd.DataFrame(predicciones_list)
            if not df_res.empty :
                df_res['Score_Recomendacion'] = df_res['Score_Recomendacion'].round(4)
                print(df_res[['Producto', 'Score_Recomendacion', 'Cantidad_Predicha', 'Meses_Historicos_Locales']].to_string(index=False))
        else:
            print("No hay predicciones para esta ubicación.")

Cargando y preparando datos...
Datos cargados. Dimensiones: (57429, 68)
Advertencia: La columna de cantidad 'CANTIDAD' no fue encontrada. Se asumirá 1 por transacción.
Datos preparados correctamente.

Generando recomendaciones por similitud...

Recomendaciones por similitud generadas.

Prediciendo cantidades para productos recomendados...


=== RESULTADOS FINALES ===
Recomendaciones con predicciones de cantidad:

--- Abejorral / Antioquia ---
                                Producto  Score_Recomendacion  Cantidad_Predicha  Meses_Historicos_Locales
                         FLETE POR ENVIO              47.1837                  1                         0
CELULAR NOTE 13 PRO + 5G 512 NEGRO 12RAM              46.1521                  1                         0
     CELULAR POCO X6 PRO 5G 512 AMARILLO              44.6954                  1                         0
         CELULAR POCO X6 PRO 5G 512 GRIS              41.4118                  7                         0
CELULAR SAMSUNG A5

In [195]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.linear_model import LinearRegression
import numpy as np
from datetime import datetime
# from dateutil.relativedelta import relativedelta
import warnings

from sklearn.metrics import mean_absolute_error, mean_squared_error

# Configuración inicial
warnings.filterwarnings('ignore')

# =============================================
# CONFIGURACIÓN DE PARÁMETROS (AJUSTAR SEGÚN DATOS)
# =============================================
excel_filename = 'basesi.xlsx'
columna_geografica = 'CIUDAD'
columna_producto = 'PRODUCTO'
columna_fecha = 'FECHA'
columna_cantidad = 'CANTIDAD'

min_meses_historia = 2
default_cantidad = 10
min_meses_para_regresion = 4
top_n_similar_locations_for_cold_start = 5
train_split_ratio = 0.8 # 80% para entrenamiento, 20% para prueba

# =============================================
# 1. CARGA Y PREPARACIÓN DE DATOS
# =============================================
print("Cargando y preparando datos...")
try:
    data_original = pd.read_excel(excel_filename, engine='openpyxl')
    print(f"Datos originales cargados. Dimensiones: {data_original.shape}")

    columnas_requeridas_base = [columna_geografica, columna_producto, columna_fecha]
    columnas_a_usar = columnas_requeridas_base.copy()

    if columna_cantidad in data_original.columns:
        columnas_a_usar.append(columna_cantidad)
    else:
        print(f"Advertencia: La columna de cantidad '{columna_cantidad}' no fue encontrada. Se asumirá 1 por transacción.")

    for col in columnas_requeridas_base:
        if col not in data_original.columns:
            raise ValueError(f"Columna requerida '{col}' no encontrada en el archivo Excel.")

    data = data_original[columnas_a_usar].copy()
    data.dropna(subset=columnas_requeridas_base, inplace=True)

    data[columna_fecha] = pd.to_datetime(data[columna_fecha], errors='coerce')
    data.dropna(subset=[columna_fecha], inplace=True)
    data['MES_ANO'] = data[columna_fecha].dt.to_period('M')

    if columna_cantidad in data.columns:
        data[columna_cantidad] = pd.to_numeric(data[columna_cantidad], errors='coerce').fillna(1)
        data[columna_cantidad] = data[columna_cantidad].apply(lambda x: max(0, float(x)))
    else:
        data[columna_cantidad] = 1.0
    
    print("Datos preparados antes de la división.")

    # DIVISIÓN DE DATOS EN ENTRENAMIENTO Y PRUEBA (TEMPORAL)
    data = data.sort_values(by=columna_fecha)
    split_point = int(len(data) * train_split_ratio)
    train_data = data.iloc[:split_point].copy()
    test_data = data.iloc[split_point:].copy()

    print(f"Datos de entrenamiento: {train_data.shape}")
    print(f"Datos de prueba: {test_data.shape}")

    if train_data.empty:
        raise ValueError("El conjunto de entrenamiento está vacío después de la división. Ajusta 'train_split_ratio' o verifica tus datos.")

except FileNotFoundError:
    print(f"Error: El archivo '{excel_filename}' no fue encontrado.")
    raise
except ValueError as ve:
    print(f"Error de valor: {ve}")
    raise
except Exception as e:
    print(f"Ocurrió un error inesperado: {e}")
    raise

# =============================================
# 2. SISTEMA DE RECOMENDACIÓN (BASADO EN train_data)
# =============================================
print("\nGenerando recomendaciones por similitud (usando datos de entrenamiento)...")
interaction_matrix_train = pd.DataFrame()
if not train_data.empty:
    interaction_matrix_train = train_data.groupby([columna_geografica, columna_producto])[columna_cantidad].sum().unstack(fill_value=0)
else:
    print("Advertencia: train_data está vacío. No se puede generar interaction_matrix_train.")

lugar_sim_df_train = pd.DataFrame() # Inicializar como DataFrame vacío
if not interaction_matrix_train.empty and interaction_matrix_train.shape[0] >= 2:
    lugar_similarity_values = cosine_similarity(interaction_matrix_train)
    lugar_sim_df_train = pd.DataFrame(lugar_similarity_values,
                                      index=interaction_matrix_train.index,
                                      columns=interaction_matrix_train.index)
elif not interaction_matrix_train.empty:
    print("Advertencia: Solo hay una ubicación en interaction_matrix_train. Similitud detallada no posible.")
    idx_train = interaction_matrix_train.index
    if len(idx_train) > 0:
        lugar_sim_df_train = pd.DataFrame(np.eye(len(idx_train)), index=idx_train, columns=idx_train)
else:
    print("Error: interaction_matrix_train está vacía. No se puede calcular similitud.")


top_similar_partners_train = {}
if not lugar_sim_df_train.empty:
    for loc_idx_sim_calc in lugar_sim_df_train.index:
        if loc_idx_sim_calc in lugar_sim_df_train.columns:
            sim_scores = lugar_sim_df_train[loc_idx_sim_calc].drop(loc_idx_sim_calc, errors='ignore').sort_values(ascending=False)
            top_similar_partners_train[loc_idx_sim_calc] = sim_scores.head(top_n_similar_locations_for_cold_start).index.tolist()
        else:
            top_similar_partners_train[loc_idx_sim_calc] = []

# Ajustamos la función para que use las matrices de entrenamiento
def obtener_recomendaciones_train(lugar_actual, n_recomendaciones=5):
    # Esta función ahora implícitamente usa lugar_sim_df_train e interaction_matrix_train
    if lugar_sim_df_train.empty or lugar_actual not in lugar_sim_df_train.index:
        return []
    if len(lugar_sim_df_train.columns) < 2 and len(lugar_sim_df_train.index) < 2:
        return []
    if lugar_actual not in lugar_sim_df_train.columns: # Asegurar que esté en columnas también
         return []

    similar_lugares_scores = lugar_sim_df_train[lugar_actual].sort_values(ascending=False)
    similar_lugares = [loc for loc in similar_lugares_scores.index if loc != lugar_actual]

    if not similar_lugares or lugar_actual not in interaction_matrix_train.index:
        return []

    productos_actuales = interaction_matrix_train.loc[lugar_actual][interaction_matrix_train.loc[lugar_actual] > 0].index.tolist()

    recomendaciones = {}
    for lugar_similar in similar_lugares:
        if lugar_similar not in interaction_matrix_train.index: continue
        productos_lugar_similar = interaction_matrix_train.loc[lugar_similar]
        productos_nuevos = productos_lugar_similar[
            (productos_lugar_similar > 0) &
            (~productos_lugar_similar.index.isin(productos_actuales))
        ].index.tolist()
        sim_score = similar_lugares_scores.get(lugar_similar, 0)
        for producto in productos_nuevos:
            recomendaciones[producto] = recomendaciones.get(producto, 0) + sim_score
    return sorted(recomendaciones.items(), key=lambda x: x[1], reverse=True)[:n_recomendaciones]

recomendaciones_por_ubicacion_train = {}
if not interaction_matrix_train.empty and not lugar_sim_df_train.empty:
    for ubicacion_idx in interaction_matrix_train.index: # Iterar solo sobre ubicaciones de entrenamiento
        recs = obtener_recomendaciones_train(ubicacion_idx)
        if recs:
            recomendaciones_por_ubicacion_train[ubicacion_idx] = recs

if not recomendaciones_por_ubicacion_train:
    print("\nNo se pudieron generar recomendaciones basadas en datos de entrenamiento.")
else:
    print("\nRecomendaciones (entrenamiento) generadas.")

# <<< INTRODUCCIÓN DEL ERROR >>>
# Intentaremos acceder a una ciudad que SÓLO existe en el conjunto de prueba
# dentro de 'lugar_sim_df_train', que fue construido SÓLO con datos de entrenamiento.
print("\n--- INTRODUCIENDO ERROR RELACIONADO CON LA DIVISIÓN DE DATOS ---")
if not test_data.empty and not train_data.empty and not lugar_sim_df_train.empty:
    ciudades_entrenamiento = train_data[columna_geografica].unique()
    ciudades_prueba_no_en_entrenamiento = test_data[~test_data[columna_geografica].isin(ciudades_entrenamiento)][columna_geografica].unique()

    if len(ciudades_prueba_no_en_entrenamiento) > 0:
        ciudad_problematica = ciudades_prueba_no_en_entrenamiento[0]
        print(f"Ciudad problemática (solo en test, no en train): '{ciudad_problematica}'")
        print(f"Intentando acceder a '{ciudad_problematica}' en 'lugar_sim_df_train' (matriz de similitud de entrenamiento)...")
        try:
            # Esto causará un KeyError si ciudad_problematica no es un índice/columna en lugar_sim_df_train
            # Lo cual es esperado si la ciudad es nueva en el período de prueba.
            sim_scores_problematicos = lugar_sim_df_train[ciudad_problematica] # <<< ESTA LÍNEA CAUSARÁ EL ERROR
            print("Acceso inesperadamente exitoso (esto no debería pasar si la ciudad es realmente nueva).")
        except KeyError as ke:
            print(f"ERROR CAPTURADO (esperado): {ke}")
            print("Este error ocurre porque la ciudad no estaba presente en los datos de entrenamiento con los que se construyó 'lugar_sim_df_train'.")
            # Para el resto del script, podríamos querer detenernos o manejarlo.
            # Por ahora, solo mostramos el error y continuamos con un diccionario vacío de predicciones para no romper todo.
            predicciones_finales = {} # Continuar con vacío para no fallar el resto
    else:
        print("No se encontraron ciudades en el conjunto de prueba que no estén en el de entrenamiento. No se puede demostrar el KeyError específico.")
        # En este caso, el script continuará, pero el error específico no se habrá activado.
        # Para garantizar el error, necesitarías datos donde algunas ciudades aparezcan *solo* en el período de prueba.
else:
    print("No hay suficientes datos en test/train o lugar_sim_df_train está vacío para demostrar el error.")


# =============================================
# 3. PREDICCIÓN DE CANTIDADES (BASADO EN train_data)
# =============================================
print("\n\nPrediciendo cantidades para productos recomendados (usando modelos de entrenamiento)...")

ventas_mensuales_df_train = pd.DataFrame()
if not train_data.empty:
    ventas_mensuales_df_train = train_data.groupby(
        [columna_geografica, columna_producto, 'MES_ANO']
    )[columna_cantidad].sum().reset_index()
    if 'MES_ANO' in ventas_mensuales_df_train.columns:
        ventas_mensuales_df_train['MES_ANO'] = ventas_mensuales_df_train['MES_ANO'].dt.to_timestamp()
else:
    print("Advertencia: train_data vacío, no se pueden calcular ventas_mensuales_df_train.")


ventas_mensuales_indexed_train = pd.DataFrame()
if not ventas_mensuales_df_train.empty:
    ventas_mensuales_indexed_train = ventas_mensuales_df_train.set_index([columna_geografica, columna_producto])
else:
    ventas_mensuales_indexed_train = pd.DataFrame(columns=['MES_ANO', columna_cantidad]).set_index(
        pd.MultiIndex.from_tuples([], names=(columna_geografica, columna_producto)))

promedio_ventas_producto_global_df_train = pd.DataFrame()
if not ventas_mensuales_df_train.empty:
    promedio_ventas_producto_global_df_train = ventas_mensuales_df_train.groupby(columna_producto, as_index=False)[columna_cantidad].mean()
    promedio_ventas_producto_global_df_train.rename(columns={columna_cantidad: 'CANTIDAD_PROMEDIO_GLOBAL'}, inplace=True)
else:
    promedio_ventas_producto_global_df_train = pd.DataFrame(columns=[columna_producto, 'CANTIDAD_PROMEDIO_GLOBAL'])


# Si el error anterior detuvo la ejecución o queremos simular que no hay predicciones si ocurrió un problema grave:
if 'predicciones_finales' not in locals(): # Si la variable no fue definida por el try-except del error
    predicciones_finales = {}
else: # Si el try-except manejó el error y asignó predicciones_finales, lo usamos.
      # Pero para este ejemplo, vamos a recalcular predicciones solo con datos de train.
      pass

# Reiniciamos predicciones_finales para que se base en recomendaciones de entrenamiento
predicciones_finales = {}


def get_avg_sales_in_locations_train(product_name, locations_list, ventas_df_local_train, geo_col, prod_col, qty_col):
    if not locations_list or ventas_df_local_train.empty:
        return None
    relevant_sales = ventas_df_local_train[
        (ventas_df_local_train[prod_col] == product_name) &
        (ventas_df_local_train[geo_col].isin(locations_list))
    ]
    if not relevant_sales.empty and qty_col in relevant_sales.columns and not relevant_sales[qty_col].empty:
        return relevant_sales[qty_col].mean()
    return None

# Iterar sobre las recomendaciones generadas a partir de datos de ENTRENAMIENTO
for ubicacion, productos_recomendados in recomendaciones_por_ubicacion_train.items():
    predicciones_finales[ubicacion] = []
    for producto, score in productos_recomendados:
        historial_local = pd.DataFrame()
        try:
            # Usar el DataFrame indexado de ENTRENAMIENTO
            if not ventas_mensuales_indexed_train.empty and (ubicacion, producto) in ventas_mensuales_indexed_train.index:
                historial_data_lookup = ventas_mensuales_indexed_train.loc[(ubicacion, producto)]
                if isinstance(historial_data_lookup, pd.Series):
                    historial_local = historial_data_lookup.to_frame().T.sort_values('MES_ANO').reset_index()
                elif isinstance(historial_data_lookup, pd.DataFrame):
                    historial_local = historial_data_lookup.sort_values('MES_ANO').reset_index()
        except KeyError:
            pass
        
        if historial_local.empty: # Asegurar estructura
             historial_local = pd.DataFrame(columns=[columna_geografica, columna_producto, 'MES_ANO', columna_cantidad])

        cantidad_predicha_float = float(default_cantidad)
        meses_historicos_locales = len(historial_local)

        if meses_historicos_locales >= min_meses_para_regresion:
            historial_local_reg = historial_local.assign(TIME_INDEX=np.arange(len(historial_local)))
            if pd.api.types.is_numeric_dtype(historial_local_reg[columna_cantidad]) and not historial_local_reg[columna_cantidad].isnull().any():
                try:
                    model = LinearRegression()
                    model.fit(historial_local_reg[['TIME_INDEX']], historial_local_reg[columna_cantidad])
                    pred = model.predict([[len(historial_local_reg)]])[0]
                    cantidad_predicha_float = pred
                    
                    p75_local = historial_local_reg[columna_cantidad].quantile(0.75)
                    if pd.notna(p75_local) and p75_local > 0 and cantidad_predicha_float > 3 * p75_local:
                        mean_local = historial_local_reg[columna_cantidad].mean()
                        std_local = historial_local_reg[columna_cantidad].std()
                        cap_val = 1.5 * p75_local
                        if pd.notna(mean_local) and pd.notna(std_local):
                             cap_val = max(cap_val, mean_local + 2 * (std_local if pd.notna(std_local) else 0))
                        if pd.notna(cap_val): cantidad_predicha_float = cap_val
                    elif (pd.isna(p75_local) or p75_local == 0) and cantidad_predicha_float > default_cantidad * 2:
                        cantidad_predicha_float = float(default_cantidad * 2)
                except Exception as reg_ex:
                    print(f"Advertencia: Falla en regresión para {ubicacion}-{producto} (train). Usando promedio. Error: {reg_ex}")
                    if not historial_local.empty and columna_cantidad in historial_local.columns and not historial_local[columna_cantidad].empty and pd.notna(historial_local[columna_cantidad].mean()):
                        cantidad_predicha_float = historial_local[columna_cantidad].mean()
            else:
                if not historial_local.empty and columna_cantidad in historial_local.columns and not historial_local[columna_cantidad].empty and pd.notna(historial_local[columna_cantidad].mean()):
                    cantidad_predicha_float = historial_local[columna_cantidad].mean()
        elif meses_historicos_locales > 0:
            if not historial_local.empty and columna_cantidad in historial_local.columns and not historial_local[columna_cantidad].empty and pd.notna(historial_local[columna_cantidad].mean()):
                cantidad_predicha_float = historial_local[columna_cantidad].mean()
        else: # Cold start
            estimacion_cold_start = None
            top_sim_locs = top_similar_partners_train.get(ubicacion, []) # Usar _train
            if top_sim_locs:
                avg_sales_sim = get_avg_sales_in_locations_train( # Usar _train
                    producto, top_sim_locs, ventas_mensuales_df_train, # Usar _train
                    columna_geografica, columna_producto, columna_cantidad
                )
                if avg_sales_sim is not None and pd.notna(avg_sales_sim):
                    estimacion_cold_start = avg_sales_sim
            if estimacion_cold_start is None or estimacion_cold_start < 1:
                if not promedio_ventas_producto_global_df_train.empty: # Usar _train
                    info_prod_gbl = promedio_ventas_producto_global_df_train[
                        promedio_ventas_producto_global_df_train[columna_producto] == producto
                    ]
                    if not info_prod_gbl.empty:
                        avg_gbl_sales = info_prod_gbl['CANTIDAD_PROMEDIO_GLOBAL'].iloc[0]
                        if pd.notna(avg_gbl_sales): estimacion_cold_start = avg_gbl_sales
            if estimacion_cold_start is not None and pd.notna(estimacion_cold_start) and estimacion_cold_start >=1:
                cantidad_predicha_float = estimacion_cold_start

        cantidad_predicha_float = max(0, cantidad_predicha_float)
        cantidad_predicha = max(1, int(round(cantidad_predicha_float)))
        predicciones_finales[ubicacion].append({
            'Producto': producto,
            'Score_Recomendacion': score,
            'Cantidad_Predicha': cantidad_predicha,
            'Meses_Historicos_Locales': meses_historicos_locales
        })

# =============================================
# 4. EVALUACIÓN DEL MODELO (SOBRE test_data)
# =============================================
print("\n\nEvaluando el modelo sobre el conjunto de prueba...")

# Lista para guardar tuplas de (cantidad_real, cantidad_predicha)
predicciones_para_evaluacion_lista = []

if test_data.empty:
    print("El conjunto de prueba está vacío. No se puede realizar la evaluación.")
else:
    # Agrupar los datos de prueba para obtener las ventas reales mensuales por ubicación y producto
    ventas_reales_test_agg = test_data.groupby(
        [columna_geografica, columna_producto, 'MES_ANO']
    )[columna_cantidad].sum().reset_index()
    ventas_reales_test_agg['MES_ANO'] = ventas_reales_test_agg['MES_ANO'].dt.to_timestamp()

    # Iterar sobre cada fila del conjunto de prueba agregado (representa una venta real en un mes)
    # O, de forma más enfocada, podrías iterar sobre ubicaciones en test_data y luego
    # sobre productos que tu sistema recomendaría para esas ubicaciones.
    # Para un cálculo de error directo sobre lo que SÍ ocurrió en test:

    print(f"Procesando {len(ventas_reales_test_agg)} registros de ventas reales en el conjunto de prueba para evaluación.")
    for index, fila_real in ventas_reales_test_agg.iterrows():
        ubicacion_eval = fila_real[columna_geografica]
        producto_eval = fila_real[columna_producto]
        cantidad_real_eval = fila_real[columna_cantidad]
        # mes_ano_eval = fila_real['MES_ANO'] # Podrías usarlo para predicciones más específicas del mes si tu modelo lo soporta

        # Solo proceder si la ubicación es conocida por el modelo de similitud (entrenado con train_data)
        if ubicacion_eval not in interaction_matrix_train.index:
            # print(f"Advertencia: Ubicación '{ubicacion_eval}' de test_data no encontrada en datos de entrenamiento. Omitiendo para evaluación de cantidad.")
            continue

        # --- Lógica de predicción de cantidad (similar a la sección 3, pero usando modelos de train) ---
        historial_local_eval = pd.DataFrame()
        try:
            if not ventas_mensuales_indexed_train.empty and (ubicacion_eval, producto_eval) in ventas_mensuales_indexed_train.index:
                historial_data_lookup_eval = ventas_mensuales_indexed_train.loc[(ubicacion_eval, producto_eval)]
                if isinstance(historial_data_lookup_eval, pd.Series):
                    historial_local_eval = historial_data_lookup_eval.to_frame().T.sort_values('MES_ANO').reset_index()
                elif isinstance(historial_data_lookup_eval, pd.DataFrame):
                    historial_local_eval = historial_data_lookup_eval.sort_values('MES_ANO').reset_index()
        except KeyError:
            pass # El producto puede no tener historial en train para esta ubicación

        if historial_local_eval.empty:
            historial_local_eval = pd.DataFrame(columns=[columna_geografica, columna_producto, 'MES_ANO', columna_cantidad])

        cantidad_predicha_eval_float = float(default_cantidad)
        meses_historicos_locales_eval = len(historial_local_eval)

        if meses_historicos_locales_eval >= min_meses_para_regresion:
            historial_local_reg_eval = historial_local_eval.assign(TIME_INDEX=np.arange(len(historial_local_eval)))
            if pd.api.types.is_numeric_dtype(historial_local_reg_eval[columna_cantidad]) and not historial_local_reg_eval[columna_cantidad].isnull().any():
                try:
                    model_eval = LinearRegression()
                    model_eval.fit(historial_local_reg_eval[['TIME_INDEX']], historial_local_reg_eval[columna_cantidad])
                    # Predicción para el "siguiente" periodo basado en el historial de train
                    pred_eval = model_eval.predict([[len(historial_local_reg_eval)]])[0]
                    cantidad_predicha_eval_float = pred_eval

                    # Aplicar lógica de capping (idéntica a la de la sección de predicción principal)
                    p75_local_eval = historial_local_reg_eval[columna_cantidad].quantile(0.75)
                    if pd.notna(p75_local_eval) and p75_local_eval > 0 and cantidad_predicha_eval_float > 3 * p75_local_eval:
                        mean_local_eval = historial_local_reg_eval[columna_cantidad].mean()
                        std_local_eval = historial_local_reg_eval[columna_cantidad].std()
                        cap_val_eval = 1.5 * p75_local_eval
                        if pd.notna(mean_local_eval) and pd.notna(std_local_eval):
                            cap_val_eval = max(cap_val_eval, mean_local_eval + 2 * (std_local_eval if pd.notna(std_local_eval) else 0))
                        if pd.notna(cap_val_eval): cantidad_predicha_eval_float = cap_val_eval
                    elif (pd.isna(p75_local_eval) or p75_local_eval == 0) and cantidad_predicha_eval_float > default_cantidad * 2:
                        cantidad_predicha_eval_float = float(default_cantidad * 2)
                except Exception as reg_ex_eval:
                    if not historial_local_eval.empty and columna_cantidad in historial_local_eval.columns and not historial_local_eval[columna_cantidad].empty and pd.notna(historial_local_eval[columna_cantidad].mean()):
                        cantidad_predicha_eval_float = historial_local_eval[columna_cantidad].mean()
            else:
                if not historial_local_eval.empty and columna_cantidad in historial_local_eval.columns and not historial_local_eval[columna_cantidad].empty and pd.notna(historial_local_eval[columna_cantidad].mean()):
                    cantidad_predicha_eval_float = historial_local_eval[columna_cantidad].mean()
        elif meses_historicos_locales_eval > 0: # Usar promedio si hay algo de historia pero no suficiente para regresión
            if not historial_local_eval.empty and columna_cantidad in historial_local_eval.columns and not historial_local_eval[columna_cantidad].empty and pd.notna(historial_local_eval[columna_cantidad].mean()):
                cantidad_predicha_eval_float = historial_local_eval[columna_cantidad].mean()
        else: # Cold start para la predicción de cantidad (producto sin historial en esa ubicación)
            estimacion_cold_start_eval = None
            top_sim_locs_eval = top_similar_partners_train.get(ubicacion_eval, [])
            if top_sim_locs_eval:
                avg_sales_sim_eval = get_avg_sales_in_locations_train(
                    producto_eval, top_sim_locs_eval, ventas_mensuales_df_train,
                    columna_geografica, columna_producto, columna_cantidad
                )
                if avg_sales_sim_eval is not None and pd.notna(avg_sales_sim_eval):
                    estimacion_cold_start_eval = avg_sales_sim_eval

            if estimacion_cold_start_eval is None or estimacion_cold_start_eval < 1: # Si no hay estimación por similitud o es muy baja
                if not promedio_ventas_producto_global_df_train.empty:
                    info_prod_gbl_eval = promedio_ventas_producto_global_df_train[
                        promedio_ventas_producto_global_df_train[columna_producto] == producto_eval
                    ]
                    if not info_prod_gbl_eval.empty:
                        avg_gbl_sales_eval = info_prod_gbl_eval['CANTIDAD_PROMEDIO_GLOBAL'].iloc[0]
                        if pd.notna(avg_gbl_sales_eval): estimacion_cold_start_eval = avg_gbl_sales_eval
            
            if estimacion_cold_start_eval is not None and pd.notna(estimacion_cold_start_eval) and estimacion_cold_start_eval >=1:
                cantidad_predicha_eval_float = estimacion_cold_start_eval
            # Si sigue siendo None, se mantendrá el default_cantidad asignado al inicio.

        cantidad_predicha_eval_final = max(1, int(round(max(0, cantidad_predicha_eval_float))))

        predicciones_para_evaluacion_lista.append({
            'ubicacion': ubicacion_eval,
            'producto': producto_eval,
            'real': cantidad_real_eval,
            'predicho': cantidad_predicha_eval_final
        })

    # --- Calcular Métricas de Error ---
    if predicciones_para_evaluacion_lista:
        df_evaluacion = pd.DataFrame(predicciones_para_evaluacion_lista)
        
        print(f"\nResumen de predicciones vs reales para evaluación ({len(df_evaluacion)} pares):")
        print(df_evaluacion.head())

        reales_np = df_evaluacion['real'].values
        predichos_np = df_evaluacion['predicho'].values

        mae = mean_absolute_error(reales_np, predichos_np)
        mse = mean_squared_error(reales_np, predichos_np)
        rmse = np.sqrt(mse)

        print(f"\n--- Métricas de Error de Predicción de Cantidad (sobre datos de prueba) ---")
        print(f"Error Absoluto Medio (MAE): {mae:.2f}")
        print(f"Error Cuadrático Medio (MSE): {mse:.2f}")
        print(f"Raíz del Error Cuadrático Medio (RMSE): {rmse:.2f}")

        # MAPE (Error Porcentual Absoluto Medio) - Cuidado con valores reales iguales a cero
        df_evaluacion_mape = df_evaluacion[df_evaluacion['real'] != 0].copy() # Evitar división por cero
        if not df_evaluacion_mape.empty:
            df_evaluacion_mape['error_abs_porcentual'] = np.abs((df_evaluacion_mape['real'] - df_evaluacion_mape['predicho']) / df_evaluacion_mape['real'])
            mape = np.mean(df_evaluacion_mape['error_abs_porcentual']) * 100
            print(f"Error Porcentual Absoluto Medio (MAPE): {mape:.2f}% (calculado sobre {len(df_evaluacion_mape)} instancias con ventas reales > 0)")
        else:
            print("MAPE no se puede calcular (no hay instancias con ventas reales > 0 en el conjunto de evaluación).")
    else:
        print("No se generaron predicciones para evaluar (posiblemente no hay datos de prueba relevantes o ubicaciones comunes).")


# =============================================
# 5. RESULTADOS FINALES (basados en modelos de entrenamiento para recomendaciones futuras)

print("\n\n=== RESULTADOS FINALES (basados en modelos de entrenamiento) ===")
if not predicciones_finales:
    print(f"No se generaron predicciones finales.")
else:
    print(f"Recomendaciones con predicciones de cantidad:")
    for ubicacion_res, predicciones_list in predicciones_finales.items():
        print(f"\n--- {ubicacion_res} ---")
        if predicciones_list:
            df_res = pd.DataFrame(predicciones_list)
            if not df_res.empty :
                df_res['Score_Recomendacion'] = df_res['Score_Recomendacion'].round(4)
                print(df_res[['Producto', 'Score_Recomendacion', 'Cantidad_Predicha', 'Meses_Historicos_Locales']].to_string(index=False))
        else:
            print("No hay predicciones para esta ubicación.")

Cargando y preparando datos...
Datos originales cargados. Dimensiones: (57429, 68)
Advertencia: La columna de cantidad 'CANTIDAD' no fue encontrada. Se asumirá 1 por transacción.
Datos preparados antes de la división.
Datos de entrenamiento: (45938, 5)
Datos de prueba: (11485, 5)

Generando recomendaciones por similitud (usando datos de entrenamiento)...

Recomendaciones (entrenamiento) generadas.

--- INTRODUCIENDO ERROR RELACIONADO CON LA DIVISIÓN DE DATOS ---
Ciudad problemática (solo en test, no en train): 'Lejanías / Meta'
Intentando acceder a 'Lejanías / Meta' en 'lugar_sim_df_train' (matriz de similitud de entrenamiento)...
ERROR CAPTURADO (esperado): 'Lejanías / Meta'
Este error ocurre porque la ciudad no estaba presente en los datos de entrenamiento con los que se construyó 'lugar_sim_df_train'.


Prediciendo cantidades para productos recomendados (usando modelos de entrenamiento)...


Evaluando el modelo sobre el conjunto de prueba...
Procesando 4717 registros de ventas reale

In [197]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.linear_model import LinearRegression
import numpy as np
from datetime import datetime
# from dateutil.relativedelta import relativedelta
import warnings
from sklearn.metrics import mean_absolute_error, mean_squared_error # Asegúrate que esta línea está

# Configuración inicial
warnings.filterwarnings('ignore')

# =============================================
# CONFIGURACIÓN DE PARÁMETROS (AJUSTAR SEGÚN DATOS)
# =============================================
excel_filename = 'basesi.xlsx'
columna_geografica = 'CIUDAD'
columna_producto = 'PRODUCTO'
columna_fecha = 'FECHA'
columna_cantidad = 'CANTIDAD' # El script manejará si no existe

min_meses_historia = 2
default_cantidad = 10
min_meses_para_regresion = 4
top_n_similar_locations_for_cold_start = 5
train_split_ratio = 0.8 # 80% para entrenamiento, 20% para prueba

# =============================================
# 1. CARGA Y PREPARACIÓN DE DATOS
# =============================================
print("Cargando y preparando datos...")
try:
    data_original = pd.read_excel(excel_filename, engine='openpyxl')
    print(f"Datos originales cargados. Dimensiones: {data_original.shape}")

    columnas_requeridas_base = [columna_geografica, columna_producto, columna_fecha]
    columnas_a_usar = columnas_requeridas_base.copy()

    cantidad_col_existe = columna_cantidad in data_original.columns
    if cantidad_col_existe:
        columnas_a_usar.append(columna_cantidad)
    else:
        print(f"Advertencia: La columna de cantidad '{columna_cantidad}' no fue encontrada. Se asumirá 1 por transacción.")

    for col in columnas_requeridas_base:
        if col not in data_original.columns:
            raise ValueError(f"Columna requerida '{col}' no encontrada en el archivo Excel.")

    data = data_original[columnas_a_usar].copy()
    data.dropna(subset=columnas_requeridas_base, inplace=True)

    data[columna_fecha] = pd.to_datetime(data[columna_fecha], errors='coerce')
    data.dropna(subset=[columna_fecha], inplace=True)
    data['MES_ANO'] = data[columna_fecha].dt.to_period('M')

    if cantidad_col_existe:
        data[columna_cantidad] = pd.to_numeric(data[columna_cantidad], errors='coerce').fillna(1)
        data[columna_cantidad] = data[columna_cantidad].apply(lambda x: max(0, float(x)))
    else:
        data[columna_cantidad] = 1.0 # Asignar 1.0 si la columna no existe
    
    print("Datos preparados antes de la división.")

    # DIVISIÓN DE DATOS EN ENTRENAMIENTO Y PRUEBA (TEMPORAL)
    data = data.sort_values(by=columna_fecha)
    split_point = int(len(data) * train_split_ratio)
    train_data = data.iloc[:split_point].copy()
    test_data = data.iloc[split_point:].copy()

    print(f"Datos de entrenamiento: {train_data.shape}")
    print(f"Datos de prueba: {test_data.shape}")

    if train_data.empty:
        raise ValueError("El conjunto de entrenamiento está vacío después de la división. Ajusta 'train_split_ratio' o verifica tus datos.")

except FileNotFoundError:
    print(f"Error: El archivo '{excel_filename}' no fue encontrado.")
    raise
except ValueError as ve:
    print(f"Error de valor: {ve}")
    raise
except Exception as e:
    print(f"Ocurrió un error inesperado: {e}")
    raise

# =============================================
# 2. SISTEMA DE RECOMENDACIÓN (BASADO EN train_data)
# =============================================
print("\nGenerando recomendaciones por similitud (usando datos de entrenamiento)...")
interaction_matrix_train = pd.DataFrame()
if not train_data.empty:
    interaction_matrix_train = train_data.groupby([columna_geografica, columna_producto])[columna_cantidad].sum().unstack(fill_value=0)
else:
    print("Advertencia: train_data está vacío. No se puede generar interaction_matrix_train.")

lugar_sim_df_train = pd.DataFrame() 
if not interaction_matrix_train.empty and interaction_matrix_train.shape[0] >= 2:
    lugar_similarity_values = cosine_similarity(interaction_matrix_train)
    lugar_sim_df_train = pd.DataFrame(lugar_similarity_values,
                                      index=interaction_matrix_train.index,
                                      columns=interaction_matrix_train.index)
elif not interaction_matrix_train.empty:
    print("Advertencia: Solo hay una o ninguna ubicación en interaction_matrix_train después del procesamiento. Similitud detallada no posible.")
    idx_train = interaction_matrix_train.index
    if len(idx_train) > 0: # Si hay al menos una ubicación
        lugar_sim_df_train = pd.DataFrame(np.eye(len(idx_train)), index=idx_train, columns=idx_train)
else:
    print("Error: interaction_matrix_train está vacía. No se puede calcular similitud.")


top_similar_partners_train = {}
if not lugar_sim_df_train.empty:
    for loc_idx_sim_calc in lugar_sim_df_train.index:
        if loc_idx_sim_calc in lugar_sim_df_train.columns:
            sim_scores = lugar_sim_df_train[loc_idx_sim_calc].drop(loc_idx_sim_calc, errors='ignore').sort_values(ascending=False)
            top_similar_partners_train[loc_idx_sim_calc] = sim_scores.head(top_n_similar_locations_for_cold_start).index.tolist()
        else:
            top_similar_partners_train[loc_idx_sim_calc] = []

def obtener_recomendaciones_train(lugar_actual, n_recomendaciones=5):
    if lugar_sim_df_train.empty or lugar_actual not in lugar_sim_df_train.index or lugar_actual not in lugar_sim_df_train.columns:
        return []
    if interaction_matrix_train.empty or lugar_actual not in interaction_matrix_train.index:
        return []
    # Handle cases with very few locations
    if len(lugar_sim_df_train.columns) < 1 or len(lugar_sim_df_train.index) < 1:
        return []


    similar_lugares_scores = lugar_sim_df_train[lugar_actual].sort_values(ascending=False)
    similar_lugares = [loc for loc in similar_lugares_scores.index if loc != lugar_actual]

    if not similar_lugares:
        return []

    productos_actuales = interaction_matrix_train.loc[lugar_actual][interaction_matrix_train.loc[lugar_actual] > 0].index.tolist()

    recomendaciones = {}
    for lugar_similar in similar_lugares:
        if lugar_similar not in interaction_matrix_train.index: continue
        productos_lugar_similar = interaction_matrix_train.loc[lugar_similar]
        productos_nuevos = productos_lugar_similar[
            (productos_lugar_similar > 0) &
            (~productos_lugar_similar.index.isin(productos_actuales))
        ].index.tolist()
        sim_score = similar_lugares_scores.get(lugar_similar, 0)
        for producto in productos_nuevos:
            recomendaciones[producto] = recomendaciones.get(producto, 0) + sim_score
    return sorted(recomendaciones.items(), key=lambda x: x[1], reverse=True)[:n_recomendaciones]

recomendaciones_por_ubicacion_train = {}
if not interaction_matrix_train.empty and not lugar_sim_df_train.empty:
    for ubicacion_idx in interaction_matrix_train.index: 
        recs = obtener_recomendaciones_train(ubicacion_idx)
        if recs:
            recomendaciones_por_ubicacion_train[ubicacion_idx] = recs

if not recomendaciones_por_ubicacion_train:
    print("\nNo se pudieron generar recomendaciones basadas en datos de entrenamiento.")
else:
    print("\nRecomendaciones (entrenamiento) generadas.")

# <<< INTRODUCCIÓN DEL ERROR >>>
print("\n--- INTRODUCIENDO ERROR RELACIONADO CON LA DIVISIÓN DE DATOS ---")
if not test_data.empty and not train_data.empty and not lugar_sim_df_train.empty:
    ciudades_entrenamiento = train_data[columna_geografica].unique()
    ciudades_prueba_no_en_entrenamiento = test_data[~test_data[columna_geografica].isin(ciudades_entrenamiento)][columna_geografica].unique()

    if len(ciudades_prueba_no_en_entrenamiento) > 0:
        ciudad_problematica = ciudades_prueba_no_en_entrenamiento[0]
        print(f"Ciudad problemática (solo en test, no en train): '{ciudad_problematica}'")
        print(f"Intentando acceder a '{ciudad_problematica}' en 'lugar_sim_df_train' (matriz de similitud de entrenamiento)...")
        try:
            sim_scores_problematicos = lugar_sim_df_train[ciudad_problematica] 
            print("Acceso inesperadamente exitoso (esto no debería pasar si la ciudad es realmente nueva).")
        except KeyError as ke:
            print(f"ERROR CAPTURADO (esperado): {ke}")
            print("Este error ocurre porque la ciudad no estaba presente en los datos de entrenamiento con los que se construyó 'lugar_sim_df_train'.")
    else:
        print("No se encontraron ciudades en el conjunto de prueba que no estén en el de entrenamiento. No se puede demostrar el KeyError específico.")
else:
    print("No hay suficientes datos en test/train o lugar_sim_df_train está vacío para demostrar el error.")


# =============================================
# 3. PREDICCIÓN DE CANTIDADES (BASADO EN train_data, PARA RECOMENDACIONES FUTURAS)
# =============================================
print("\n\nPrediciendo cantidades para productos recomendados (usando modelos de entrenamiento)...")

ventas_mensuales_df_train = pd.DataFrame()
if not train_data.empty:
    ventas_mensuales_df_train = train_data.groupby(
        [columna_geografica, columna_producto, 'MES_ANO']
    )[columna_cantidad].sum().reset_index()
    if 'MES_ANO' in ventas_mensuales_df_train.columns:
        ventas_mensuales_df_train['MES_ANO'] = ventas_mensuales_df_train['MES_ANO'].dt.to_timestamp()
else:
    print("Advertencia: train_data vacío, no se pueden calcular ventas_mensuales_df_train.")


ventas_mensuales_indexed_train = pd.DataFrame()
if not ventas_mensuales_df_train.empty:
    ventas_mensuales_indexed_train = ventas_mensuales_df_train.set_index([columna_geografica, columna_producto])
else:
    ventas_mensuales_indexed_train = pd.DataFrame(columns=['MES_ANO', columna_cantidad]).set_index(
        pd.MultiIndex.from_tuples([], names=(columna_geografica, columna_producto)))

promedio_ventas_producto_global_df_train = pd.DataFrame()
if not ventas_mensuales_df_train.empty:
    promedio_ventas_producto_global_df_train = ventas_mensuales_df_train.groupby(columna_producto, as_index=False)[columna_cantidad].mean()
    promedio_ventas_producto_global_df_train.rename(columns={columna_cantidad: 'CANTIDAD_PROMEDIO_GLOBAL'}, inplace=True)
else:
    promedio_ventas_producto_global_df_train = pd.DataFrame(columns=[columna_producto, 'CANTIDAD_PROMEDIO_GLOBAL'])

predicciones_finales = {}

def get_avg_sales_in_locations_train(product_name, locations_list, ventas_df_local_train, geo_col, prod_col, qty_col):
    if not locations_list or ventas_df_local_train.empty:
        return None
    relevant_sales = ventas_df_local_train[
        (ventas_df_local_train[prod_col] == product_name) &
        (ventas_df_local_train[geo_col].isin(locations_list))
    ]
    if not relevant_sales.empty and qty_col in relevant_sales.columns and not relevant_sales[qty_col].empty:
        return relevant_sales[qty_col].mean()
    return None

# Iterar sobre las recomendaciones generadas a partir de datos de ENTRENAMIENTO
for ubicacion, productos_recomendados in recomendaciones_por_ubicacion_train.items():
    predicciones_finales[ubicacion] = []
    for producto, score in productos_recomendados:
        historial_local = pd.DataFrame()
        try:
            if not ventas_mensuales_indexed_train.empty and (ubicacion, producto) in ventas_mensuales_indexed_train.index:
                historial_data_lookup = ventas_mensuales_indexed_train.loc[(ubicacion, producto)]
                if isinstance(historial_data_lookup, pd.Series):
                    historial_local = historial_data_lookup.to_frame().T.sort_values('MES_ANO').reset_index()
                elif isinstance(historial_data_lookup, pd.DataFrame):
                    historial_local = historial_data_lookup.sort_values('MES_ANO').reset_index()
        except KeyError:
            pass
        
        if historial_local.empty: 
            historial_local = pd.DataFrame(columns=[columna_geografica, columna_producto, 'MES_ANO', columna_cantidad])

        cantidad_predicha_float = float(default_cantidad)
        # meses_historicos_locales = len(historial_local) # No se usa en la salida final

        if len(historial_local) >= min_meses_para_regresion: # Usar len(historial_local) directamente
            historial_local_reg = historial_local.assign(TIME_INDEX=np.arange(len(historial_local)))
            if pd.api.types.is_numeric_dtype(historial_local_reg[columna_cantidad]) and not historial_local_reg[columna_cantidad].isnull().any():
                try:
                    model = LinearRegression()
                    model.fit(historial_local_reg[['TIME_INDEX']], historial_local_reg[columna_cantidad])
                    pred = model.predict([[len(historial_local_reg)]])[0]
                    cantidad_predicha_float = pred
                    
                    p75_local = historial_local_reg[columna_cantidad].quantile(0.75)
                    if pd.notna(p75_local) and p75_local > 0 and cantidad_predicha_float > 3 * p75_local:
                        mean_local = historial_local_reg[columna_cantidad].mean()
                        std_local = historial_local_reg[columna_cantidad].std()
                        cap_val = 1.5 * p75_local
                        if pd.notna(mean_local) and pd.notna(std_local):
                                cap_val = max(cap_val, mean_local + 2 * (std_local if pd.notna(std_local) else 0))
                        if pd.notna(cap_val): cantidad_predicha_float = cap_val
                    elif (pd.isna(p75_local) or p75_local == 0) and cantidad_predicha_float > default_cantidad * 2:
                        cantidad_predicha_float = float(default_cantidad * 2)
                except Exception as reg_ex:
                    # print(f"Advertencia: Falla en regresión para {ubicacion}-{producto} (train). Usando promedio. Error: {reg_ex}")
                    if not historial_local.empty and columna_cantidad in historial_local.columns and not historial_local[columna_cantidad].empty and pd.notna(historial_local[columna_cantidad].mean()):
                        cantidad_predicha_float = historial_local[columna_cantidad].mean()
            else:
                if not historial_local.empty and columna_cantidad in historial_local.columns and not historial_local[columna_cantidad].empty and pd.notna(historial_local[columna_cantidad].mean()):
                    cantidad_predicha_float = historial_local[columna_cantidad].mean()
        elif len(historial_local) > 0: # Usar len(historial_local)
            if not historial_local.empty and columna_cantidad in historial_local.columns and not historial_local[columna_cantidad].empty and pd.notna(historial_local[columna_cantidad].mean()):
                cantidad_predicha_float = historial_local[columna_cantidad].mean()
        else: # Cold start
            estimacion_cold_start = None
            top_sim_locs = top_similar_partners_train.get(ubicacion, []) 
            if top_sim_locs:
                avg_sales_sim = get_avg_sales_in_locations_train( 
                    producto, top_sim_locs, ventas_mensuales_df_train, 
                    columna_geografica, columna_producto, columna_cantidad
                )
                if avg_sales_sim is not None and pd.notna(avg_sales_sim):
                    estimacion_cold_start = avg_sales_sim
            if estimacion_cold_start is None or estimacion_cold_start < 1:
                if not promedio_ventas_producto_global_df_train.empty: 
                    info_prod_gbl = promedio_ventas_producto_global_df_train[
                        promedio_ventas_producto_global_df_train[columna_producto] == producto
                    ]
                    if not info_prod_gbl.empty:
                        avg_gbl_sales = info_prod_gbl['CANTIDAD_PROMEDIO_GLOBAL'].iloc[0]
                        if pd.notna(avg_gbl_sales): estimacion_cold_start = avg_gbl_sales
            if estimacion_cold_start is not None and pd.notna(estimacion_cold_start) and estimacion_cold_start >=1:
                cantidad_predicha_float = estimacion_cold_start

        cantidad_predicha_float = max(0, cantidad_predicha_float)
        cantidad_predicha = max(1, int(round(cantidad_predicha_float)))
        predicciones_finales[ubicacion].append({
            'Producto': producto,
            'Score_Recomendacion': score,
            'Cantidad_Predicha': cantidad_predicha
            # 'Meses_Historicos_Locales': meses_historicos_locales # Eliminado de la salida
        })

# =============================================
# 4. EVALUACIÓN DEL MODELO (SOBRE test_data)
# =============================================
print("\n\nEvaluando el modelo sobre el conjunto de prueba...")

predicciones_para_evaluacion_lista = []
registros_test_procesados = 0
registros_test_omitidos_ubicacion_desconocida = 0

if test_data.empty:
    print("El conjunto de prueba está vacío. No se puede realizar la evaluación.")
else:
    ventas_reales_test_agg = test_data.groupby(
        [columna_geografica, columna_producto, 'MES_ANO']
    )[columna_cantidad].sum().reset_index()
    ventas_reales_test_agg['MES_ANO'] = ventas_reales_test_agg['MES_ANO'].dt.to_timestamp()

    total_registros_test_agg = len(ventas_reales_test_agg)
    print(f"Procesando {total_registros_test_agg} registros de ventas reales agregados en el conjunto de prueba para evaluación.")

    for index, fila_real in ventas_reales_test_agg.iterrows():
        ubicacion_eval = fila_real[columna_geografica]
        producto_eval = fila_real[columna_producto]
        cantidad_real_eval = fila_real[columna_cantidad]

        if ubicacion_eval not in interaction_matrix_train.index:
            registros_test_omitidos_ubicacion_desconocida += 1
            continue # Saltar esta fila si la ubicación no estaba en los datos de entrenamiento

        registros_test_procesados += 1
        # --- Lógica de predicción de cantidad (idéntica a la sección 3) ---
        historial_local_eval = pd.DataFrame()
        try:
            if not ventas_mensuales_indexed_train.empty and (ubicacion_eval, producto_eval) in ventas_mensuales_indexed_train.index:
                historial_data_lookup_eval = ventas_mensuales_indexed_train.loc[(ubicacion_eval, producto_eval)]
                if isinstance(historial_data_lookup_eval, pd.Series):
                    historial_local_eval = historial_data_lookup_eval.to_frame().T.sort_values('MES_ANO').reset_index()
                elif isinstance(historial_data_lookup_eval, pd.DataFrame):
                    historial_local_eval = historial_data_lookup_eval.sort_values('MES_ANO').reset_index()
        except KeyError:
            pass
        
        if historial_local_eval.empty:
            historial_local_eval = pd.DataFrame(columns=[columna_geografica, columna_producto, 'MES_ANO', columna_cantidad])

        cantidad_predicha_eval_float = float(default_cantidad)
        # meses_historicos_locales_eval = len(historial_local_eval) # No se usa

        if len(historial_local_eval) >= min_meses_para_regresion:
            historial_local_reg_eval = historial_local_eval.assign(TIME_INDEX=np.arange(len(historial_local_eval)))
            if pd.api.types.is_numeric_dtype(historial_local_reg_eval[columna_cantidad]) and not historial_local_reg_eval[columna_cantidad].isnull().any():
                try:
                    model_eval = LinearRegression()
                    model_eval.fit(historial_local_reg_eval[['TIME_INDEX']], historial_local_reg_eval[columna_cantidad])
                    pred_eval = model_eval.predict([[len(historial_local_reg_eval)]])[0]
                    cantidad_predicha_eval_float = pred_eval
                    
                    p75_local_eval = historial_local_reg_eval[columna_cantidad].quantile(0.75)
                    if pd.notna(p75_local_eval) and p75_local_eval > 0 and cantidad_predicha_eval_float > 3 * p75_local_eval:
                        mean_local_eval = historial_local_reg_eval[columna_cantidad].mean()
                        std_local_eval = historial_local_reg_eval[columna_cantidad].std()
                        cap_val_eval = 1.5 * p75_local_eval
                        if pd.notna(mean_local_eval) and pd.notna(std_local_eval):
                            cap_val_eval = max(cap_val_eval, mean_local_eval + 2 * (std_local_eval if pd.notna(std_local_eval) else 0))
                        if pd.notna(cap_val_eval): cantidad_predicha_eval_float = cap_val_eval
                    elif (pd.isna(p75_local_eval) or p75_local_eval == 0) and cantidad_predicha_eval_float > default_cantidad * 2:
                        cantidad_predicha_eval_float = float(default_cantidad * 2)
                except Exception as reg_ex_eval:
                    if not historial_local_eval.empty and columna_cantidad in historial_local_eval.columns and not historial_local_eval[columna_cantidad].empty and pd.notna(historial_local_eval[columna_cantidad].mean()):
                        cantidad_predicha_eval_float = historial_local_eval[columna_cantidad].mean()
            else:
                if not historial_local_eval.empty and columna_cantidad in historial_local_eval.columns and not historial_local_eval[columna_cantidad].empty and pd.notna(historial_local_eval[columna_cantidad].mean()):
                    cantidad_predicha_eval_float = historial_local_eval[columna_cantidad].mean()
        elif len(historial_local_eval) > 0:
            if not historial_local_eval.empty and columna_cantidad in historial_local_eval.columns and not historial_local_eval[columna_cantidad].empty and pd.notna(historial_local_eval[columna_cantidad].mean()):
                cantidad_predicha_eval_float = historial_local_eval[columna_cantidad].mean()
        else: 
            estimacion_cold_start_eval = None
            top_sim_locs_eval = top_similar_partners_train.get(ubicacion_eval, [])
            if top_sim_locs_eval:
                avg_sales_sim_eval = get_avg_sales_in_locations_train(
                    producto_eval, top_sim_locs_eval, ventas_mensuales_df_train,
                    columna_geografica, columna_producto, columna_cantidad
                )
                if avg_sales_sim_eval is not None and pd.notna(avg_sales_sim_eval):
                    estimacion_cold_start_eval = avg_sales_sim_eval
            if estimacion_cold_start_eval is None or estimacion_cold_start_eval < 1:
                if not promedio_ventas_producto_global_df_train.empty:
                    info_prod_gbl_eval = promedio_ventas_producto_global_df_train[
                        promedio_ventas_producto_global_df_train[columna_producto] == producto_eval
                    ]
                    if not info_prod_gbl_eval.empty:
                        avg_gbl_sales_eval = info_prod_gbl_eval['CANTIDAD_PROMEDIO_GLOBAL'].iloc[0]
                        if pd.notna(avg_gbl_sales_eval): estimacion_cold_start_eval = avg_gbl_sales_eval
            if estimacion_cold_start_eval is not None and pd.notna(estimacion_cold_start_eval) and estimacion_cold_start_eval >=1:
                cantidad_predicha_eval_float = estimacion_cold_start_eval

        cantidad_predicha_eval_final = max(1, int(round(max(0, cantidad_predicha_eval_float))))
        predicciones_para_evaluacion_lista.append({
            'real': cantidad_real_eval,
            'predicho': cantidad_predicha_eval_final
        })

    print(f"Registros de prueba procesados para evaluación: {registros_test_procesados}")
    print(f"Registros de prueba omitidos (ubicación no en train data): {registros_test_omitidos_ubicacion_desconocida}")

    if predicciones_para_evaluacion_lista:
        df_evaluacion = pd.DataFrame(predicciones_para_evaluacion_lista)
        
        print(f"\nSe generaron {len(df_evaluacion)} pares de (real, predicho) para la evaluación.")
        # print("Primeros pares para revisión:")
        # print(df_evaluacion.head()) # Descomentar si quieres ver los pares

        reales_np = df_evaluacion['real'].values
        predichos_np = df_evaluacion['predicho'].values

        mae = mean_absolute_error(reales_np, predichos_np)
        mse = mean_squared_error(reales_np, predichos_np)
        rmse = np.sqrt(mse)

        print(f"\n--- Métricas de Error de Predicción de Cantidad (sobre {len(df_evaluacion)} pares evaluados) ---")
        print(f"Error Absoluto Medio (MAE): {mae:.2f}")
        print(f"Error Cuadrático Medio (MSE): {mse:.2f}")
        print(f"Raíz del Error Cuadrático Medio (RMSE): {rmse:.2f}")

        df_evaluacion_mape = df_evaluacion[df_evaluacion['real'] != 0].copy()
        if not df_evaluacion_mape.empty:
            df_evaluacion_mape['error_abs_porcentual'] = np.abs((df_evaluacion_mape['real'] - df_evaluacion_mape['predicho']) / df_evaluacion_mape['real'])
            mape = np.mean(df_evaluacion_mape['error_abs_porcentual']) * 100
            print(f"Error Porcentual Absoluto Medio (MAPE): {mape:.2f}% (calculado sobre {len(df_evaluacion_mape)} instancias con ventas reales > 0)")
        else:
            print("MAPE no se puede calcular (no hay instancias evaluadas con ventas reales > 0).")
    else:
        print("\nNo se generaron pares de (real, predicho) para la evaluación. ")
        print("Esto puede ocurrir si ninguna de las ubicaciones en los datos de prueba estaba presente en los datos de entrenamiento,")
        print("o si no se pudo generar ninguna predicción válida para los datos de prueba.")

# =============================================
# 5. RESULTADOS FINALES (basados en modelos de entrenamiento para recomendaciones futuras)
# =============================================
print("\n\n=== RESULTADOS FINALES (basados en modelos de entrenamiento) ===")
if not predicciones_finales:
    print(f"No se generaron predicciones finales.")
else:
    print(f"Recomendaciones con predicciones de cantidad:")
    for ubicacion_res, predicciones_list in predicciones_finales.items():
        print(f"\n--- {ubicacion_res} ---")
        if predicciones_list:
            df_res = pd.DataFrame(predicciones_list)
            if not df_res.empty :
                df_res['Score_Recomendacion'] = df_res['Score_Recomendacion'].round(4)
                # Se elimina 'Meses_Historicos_Locales' de esta salida
                print(df_res[['Producto', 'Score_Recomendacion', 'Cantidad_Predicha']].to_string(index=False))
        else:
            print("No hay predicciones para esta ubicación.")

Cargando y preparando datos...
Datos originales cargados. Dimensiones: (57429, 68)
Advertencia: La columna de cantidad 'CANTIDAD' no fue encontrada. Se asumirá 1 por transacción.
Datos preparados antes de la división.
Datos de entrenamiento: (45938, 5)
Datos de prueba: (11485, 5)

Generando recomendaciones por similitud (usando datos de entrenamiento)...

Recomendaciones (entrenamiento) generadas.

--- INTRODUCIENDO ERROR RELACIONADO CON LA DIVISIÓN DE DATOS ---
Ciudad problemática (solo en test, no en train): 'Lejanías / Meta'
Intentando acceder a 'Lejanías / Meta' en 'lugar_sim_df_train' (matriz de similitud de entrenamiento)...
ERROR CAPTURADO (esperado): 'Lejanías / Meta'
Este error ocurre porque la ciudad no estaba presente en los datos de entrenamiento con los que se construyó 'lugar_sim_df_train'.


Prediciendo cantidades para productos recomendados (usando modelos de entrenamiento)...


Evaluando el modelo sobre el conjunto de prueba...
Procesando 4717 registros de ventas reale